Buy a leetcode cn account from taobao, create a new file to mark down ms problems and leetcode plus problems.

253. 会议室 II

给你一个会议时间安排的数组 intervals ，每个会议时间都会包括开始和结束的时间 intervals[i] = [starti, endi] ，返回 所需会议室的最小数量 。

 

示例 1：

输入：intervals = [[0,30],[5,10],[15,20]]
输出：2

示例 2：

输入：intervals = [[7,10],[2,4]]
输出：1

 

提示：

    1 <= intervals.length <= 104
    0 <= starti < endi <= 106



理解成上下车问题会比较容易解决，不用在意是谁上车还是下车，只需要注意什么时候上下车就可以。 以第一个示例来说：

↑    ↑    ↓     ↑      ↓             ↓
0----5----10----15-----20-----------30-->

这样可以把上车和下车的时间分成两组，通过两个指针滑动的方式，判断同时在车上的最大数就可以了。


思路

这个问题与公司员工每天可能面临的问题很相似。

假设你在一家公司的 IT 部门工作，你的工作职责之一是为办公室全天举行的会议提供会议室。

办公室有多个会议室,你希望能合理地使用它们。不让人们等待，给每组员工一个按时开会的空间。

另一方面，除非必要，你也不想占用过多会议室。如果会议互相冲突，在不同会议室安排会议；否则的话，就尽量少使用会议室。你该怎么做？

刚刚提出的是办公室中一个常见的场景，在一天天中，给定了每个会议开始和结束的时间，你需要给不同的团队安排会议室。

让我们换个角度，从一组想要开会但还没有分配会议室的人员的角度来看这个问题。他们会怎么做？

    这组人会依次查看会议室，检查是否有任何会议室是空闲的。如果他们找到一间空闲的会议室，就会在那个房间里开始开会。否则,他们会等待一个会议室空出来。一旦会议室空出来,他们就会占用它。

这就是我们将遵循的基本方法。可以说（但不确切），这是一个模拟的过程。在最坏的情况下，我们可以为每个会议分配一个新房间，但这显然不是最优的，除非它们相互冲突。

    我们需要高效地判断当前会议是否有房间可用，并只在分配的房间目前没有空闲时才分配新房间。

来看看基于上述讨论的方法一。
方法一：优先队列

我们无法按任意顺序处理给定的会议。处理会议的最基本方式是按其 开始时间 顺序排序，这也是我们采取的顺序。这就是我们将遵循的顺序。毕竟，在担心下午5：00的会议之前，你肯定应该先安排上午9：00的会议，不是吗？

让我们对一个示例问题进行一次模拟，来分析我们的算法应该能有效地执行哪些操作。

考虑下面的会议时间 (1, 10), (2, 7), (3, 19), (8, 12), (10, 20), (11, 30) 。前一部分表示会议开始时间，后一部分表示结束时间。按照会议开始时间顺序考虑。图一展示了前三个会议，每个会议都由于冲突而需要新房间。

pic-1

后面的三个会议开始占用现有的房间。然而，最后的会议需要一个新房间。总而言之，我们需要四个房间来容纳所有会议。

pic-2

排序过程很容易，但对每个会议，我们如何高效地找出是否有房间可用？任意时刻，我们都有多个可能占用的房间，只要我们能在有新会议需要时就找到一个空闲房间，我们并不需要关心到底有哪些房间是空闲的。

一个朴素的方法是，每当有新会议时，就遍历所有房间，查看是否有空闲房间。

    但是,通过使用优先队列（或最小堆）堆数据结构,我们可以做得更好。

我们可以将所有房间保存在最小堆中,堆中的键值是会议的结束时间，而不用手动迭代已分配的每个房间并检查房间是否可用。

这样，每当我们想要检查有没有 任何 房间是空的，只需要检查最小堆堆顶的元素，它是最先开完会腾出房间的。

如果堆顶的元素的房间并不空闲，那么其他所有房间都不空闲。这样，我们就可以直接开一个新房间。

在进入程序实现之前，先来看一遍算法的流程。

算法

    按照 开始时间 对会议进行排序。

    初始化一个新的 最小堆，将第一个会议的结束时间加入到堆中。我们只需要记录会议的结束时间，告诉我们什么时候房间会空。

    对每个会议，检查堆的最小元素（即堆顶部的房间）是否空闲。
        若房间空闲，则从堆顶拿出该元素，将其改为我们处理的会议的结束时间，加回到堆中。
        若房间不空闲。开新房间，并加入到堆中。

    处理完所有会议后，堆的大小即为开的房间数量。这就是容纳这些会议需要的最小房间数。

让我们尽量不要看下面的答案。

class Solution:
    def minMeetingRooms(self, intervals: List[List[int]]) -> int:

        # If there is no meeting to schedule then no room needs to be allocated.
        if not intervals:
            return 0

        # The heap initialization
        free_rooms = []

        # Sort the meetings in increasing order of their start time.
        intervals.sort(key= lambda x: x[0])

        # Add the first meeting. We have to give a new room to the first meeting.
        heapq.heappush(free_rooms, intervals[0][1])

        # For all the remaining meeting rooms
        for i in intervals[1:]:

            # If the room due to free up the earliest is free, assign that room to this meeting.
            if free_rooms[0] <= i[0]:
                heapq.heappop(free_rooms)

            # If a new room is to be assigned, then also we add to the heap,
            # If an old room is allocated, then also we have to add to the heap with updated end time.
            heapq.heappush(free_rooms, i[1])

        # The size of the heap tells us the minimum rooms required for all the meetings.
        return len(free_rooms)

复杂度分析

    时间复杂度：O(Nlog⁡N)O(N\log N)O(NlogN) 。
        时间开销主要有两部分。第一部分是数组的 排序 过程，消耗 O(Nlog⁡N)O(N\log N)O(NlogN) 的时间。数组中有 NNN 个元素。
        接下来是 最小堆 占用的时间。在最坏的情况下，全部 NNN 个会议都会互相冲突。在任何情况下，我们都要向堆执行 NNN 次插入操作。在最坏的情况下，我们要对堆进行 NNN 次查找并删除最小值操作。总的时间复杂度为 (NlogN)(NlogN)(NlogN) ，因为查找并删除最小值操作只消耗 O(log⁡N)O(\log N)O(logN) 的时间。

    空间复杂度：O(N)O(N)O(N) 。额外空间用于建立 最小堆 。在最坏的情况下，堆需要容纳全部 NNN 个元素。因此空间复杂度为 O(N)O(N)O(N) 。

方法二：有序化

思路

提供给我们的会议时间可以确定一天中所有事件的时间顺序。我们拿到了每个会议的开始和结束时间，这有助于我们定义此顺序。

根据会议的开始时间来安排会议有助于我们了解这些会议的自然顺序。然而，仅仅知道会议的开始时间，还不足以告诉我们会议的持续时间。我们还需要按照结束时间排序会议，因为一个“会议结束”事件告诉我们必然有对应的“会议开始”事件，更重要的是，“会议结束”事件可以告诉我们，一个之前被占用的会议室现在空闲了。

一个会议由其开始和结束时间定义。然而，在本算法中，我们需要 分别 处理开始时间和结束时间。这乍一听可能不太合理，毕竟开始和结束时间都是会议的一部分，如果我们将两个属性分离并分别处理，会议自身的身份就消失了。但是，这样做其实是可取的，因为：

    当我们遇到“会议结束”事件时，意味着一些较早开始的会议已经结束。我们并不关心到底是哪个会议结束。我们所需要的只是 一些 会议结束,从而提供一个空房间。

考虑上一方法中使用的案例。 要考虑的会议为：(1, 10), (2, 7), (3, 19), (8, 12), (10, 20), (11, 30) 。像之前那样，第一张图说明前三个会议彼此冲突，需要分别分配房间。

pic-3

后两张图处理剩下的会议，可以看到，我们可以复用一些已有的会议室。最终的结果是相同的，我们一共需要4个会议室，这是最优的结果。

pic-4

pic-5

算法

    分别将开始时间和结束时间存进两个数组。
    分别对开始时间和结束时间进行排序。请注意，这将打乱开始时间和结束时间的原始对应关系。它们将被分别处理。
    考虑两个指针：s_ptr 和 e_ptr ，分别代表开始指针和结束指针。开始指针遍历每个会议，结束指针帮助我们跟踪会议是否结束。
    当考虑 s_ptr 指向的特定会议时，检查该开始时间是否大于 e_ptr 指向的会议。若如此，则说明 s_ptr 开始时，已经有会议结束。于是我们可以重用房间。否则，我们就需要开新房间。
    若有会议结束，换而言之，start[s_ptr] >= end[e_ptr] ，则自增 e_ptr 。
    重复这一过程，直到 s_ptr 处理完所有会议。

让我们尽量不要看下面的答案。

class Solution:
    def minMeetingRooms(self, intervals: List[List[int]]) -> int:

        # If there are no meetings, we don't need any rooms.
        if not intervals:
            return 0

        used_rooms = 0

        # Separate out the start and the end timings and sort them individually.
        start_timings = sorted([i[0] for i in intervals])
        end_timings = sorted(i[1] for i in intervals)
        L = len(intervals)

        # The two pointers in the algorithm: e_ptr and s_ptr.
        end_pointer = 0
        start_pointer = 0

        # Until all the meetings have been processed
        while start_pointer < L:
            # If there is a meeting that has ended by the time the meeting at `start_pointer` starts
            if start_timings[start_pointer] >= end_timings[end_pointer]:
                # Free up a room and increment the end_pointer.
                used_rooms -= 1
                end_pointer += 1

            # We do this irrespective of whether a room frees up or not.
            # If a room got free, then this used_rooms += 1 wouldn't have any effect. used_rooms would
            # remain the same in that case. If no room was free, then this would increase used_rooms
            used_rooms += 1
            start_pointer += 1

        return used_rooms

复杂度分析

    时间复杂度: O(Nlog⁡N)O(N\log N)O(NlogN) 。我们所做的只是将 开始时间 和 结束时间 两个数组分别进行排序。每个数组有 NNN 个元素，因为有 NNN 个时间间隔。

    空间复杂度: O(N)O(N)O(N) 。我们建立了两个 NNN 大小的数组。分别用于记录会议的开始时间和结束时间。

作者：LeetCode
链接：https://leetcode.cn/problems/meeting-rooms-ii/solution/hui-yi-shi-ii-by-leetcode/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

解题思路

题目就是要统计同一时刻进行的最大会议的数量
我们可以把所有的开始时间和结束时间放在一起排序，
用cur表示当前进行的会议数量，遍历排序后的时间数组
如果是开始时间，cur加1，如果是结束时间，cur减一
在遍历的过程中，cur出现的最大值就是需要的房间数。
代码

class Solution:
    def minMeetingRooms(self, intervals: List[List[int]]) -> int:
        events = [(iv[0], 1) for iv in intervals] + [(iv[1], -1) for iv in intervals]
        events.sort()
        ans = cur = 0
        for _, e in events:
            cur += e
            ans = max(ans, cur)
        return ans

作者：loick
链接：https://leetcode.cn/problems/meeting-rooms-ii/solution/tong-ji-tong-shi-jin-xing-de-hui-yi-by-loick/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

解题思路

求某一时刻所需会议室的最大值，可以用差分来做
代码

class Solution:
    def minMeetingRooms(self, intervals: List[List[int]]) -> int:
        diff = defaultdict(int)
        for start, end in intervals:
            diff[start] += 1
            diff[end] -= 1
        cur = ans = 0
        for _, v in sorted(diff.items()):
            cur += v
            ans = max(ans, cur)
        return ans

作者：himymBen
链接：https://leetcode.cn/problems/meeting-rooms-ii/solution/python-chafenshuzu-by-himymben-9ekq/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

348. 设计井字棋

请在 n × n 的棋盘上，实现一个判定井字棋（Tic-Tac-Toe）胜负的神器，判断每一次玩家落子后，是否有胜出的玩家。

在这个井字棋游戏中，会有 2 名玩家，他们将轮流在棋盘上放置自己的棋子。

在实现这个判定器的过程中，你可以假设以下这些规则一定成立：

      1. 每一步棋都是在棋盘内的，并且只能被放置在一个空的格子里；

      2. 一旦游戏中有一名玩家胜出的话，游戏将不能再继续；

      3. 一个玩家如果在同一行、同一列或者同一斜对角线上都放置了自己的棋子，那么他便获得胜利。

示例:

给定棋盘边长 n = 3, 玩家 1 的棋子符号是 "X"，玩家 2 的棋子符号是 "O"。

TicTacToe toe = new TicTacToe(3);

toe.move(0, 0, 1); -> 函数返回 0 (此时，暂时没有玩家赢得这场对决)
|X| | |
| | | |    // 玩家 1 在 (0, 0) 落子。
| | | |

toe.move(0, 2, 2); -> 函数返回 0 (暂时没有玩家赢得本场比赛)
|X| |O|
| | | |    // 玩家 2 在 (0, 2) 落子。
| | | |

toe.move(2, 2, 1); -> 函数返回 0 (暂时没有玩家赢得比赛)
|X| |O|
| | | |    // 玩家 1 在 (2, 2) 落子。
| | |X|

toe.move(1, 1, 2); -> 函数返回 0 (暂没有玩家赢得比赛)
|X| |O|
| |O| |    // 玩家 2 在 (1, 1) 落子。
| | |X|

toe.move(2, 0, 1); -> 函数返回 0 (暂无玩家赢得比赛)
|X| |O|
| |O| |    // 玩家 1 在 (2, 0) 落子。
|X| |X|

toe.move(1, 0, 2); -> 函数返回 0 (没有玩家赢得比赛)
|X| |O|
|O|O| |    // 玩家 2 在 (1, 0) 落子.
|X| |X|

toe.move(2, 1, 1); -> 函数返回 1 (此时，玩家 1 赢得了该场比赛)
|X| |O|
|O|O| |    // 玩家 1 在 (2, 1) 落子。
|X|X|X|

 

进阶:
您有没有可能将每一步的 move() 操作优化到比 O(n2) 更快吗?


python 哈希表

只需要维护每一行、每一列、对角线上的 'X' 和 'O' 的个数，判断是否等于 n 即可。

特别的，还可以只维护两者之差。

class TicTacToe:

    def __init__(self, n: int):
        self.n = n
        self.r = [0]*n
        self.c = [0]*n
        self.d1 = 0
        self.d2 = 0

    def move(self, row: int, col: int, player: int) -> int:
        val = 1 if player == 1 else -1
        self.r[row] += val
        self.c[col] += val
        self.d1 += val*(row+col == self.n-1)
        self.d2 += val*(row == col)
        for x in (self.r[row], self.c[col], self.d1, self.d2):
            if x == val*self.n:
                return player
        return 0


解题思路

为两个选手分别设置参数，每次操作中更新对应参数，如果满足胜利条件，则直接返回胜者

其中，行和列有n个，所以分别用哈希表来记录；对角线和反对角线只有一条，所以只要一个参数就行
代码

class TicTacToe:

    def __init__(self, n: int):
        self.n = n
        self.col1, self.row1, self.diag1, self.antidiag1 = defaultdict(int),defaultdict(int),0,0
        self.col2, self.row2, self.diag2, self.antidiag2 = defaultdict(int),defaultdict(int),0,0

    def move(self, row: int, col: int, player: int) -> int:
        if player == 1:
            self.col1[col] += 1
            self.row1[row] += 1
            if row == col:
                self.diag1 += 1
            if row + col == self.n - 1:
                self.antidiag1 += 1
            if self.col1[col] == self.n or self.row1[row] == self.n or self.diag1 == self.n or self.antidiag1 == self.n:
                return 1
        else:
            self.col2[col] += 1
            self.row2[row] += 1
            if row == col:
                self.diag2 += 1
            if row + col == self.n - 1:
                self.antidiag2 += 1
            if self.col2[col] == self.n or self.row2[row] == self.n or self.diag2 == self.n or self.antidiag2 == self.n:
                return 2
        return 0

作者：bluegreenred
链接：https://leetcode.cn/problems/design-tic-tac-toe/solution/348-she-ji-jing-zi-qi-by-bluegreenred-5aoo/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

277. 搜寻名人

假设你是一个专业的狗仔，参加了一个 n 人派对，其中每个人被从 0 到 n - 1 标号。在这个派对人群当中可能存在一位 “名人”。所谓 “名人” 的定义是：其他所有 n - 1 个人都认识他/她，而他/她并不认识其他任何人。

现在你想要确认这个 “名人” 是谁，或者确定这里没有 “名人”。而你唯一能做的就是问诸如 “A 你好呀，请问你认不认识 B呀？” 的问题，以确定 A 是否认识 B。你需要在（渐近意义上）尽可能少的问题内来确定这位 “名人” 是谁（或者确定这里没有 “名人”）。

在本题中，你可以使用辅助函数 bool knows(a, b) 获取到 A 是否认识 B。请你来实现一个函数 int findCelebrity(n)。

派对最多只会有一个 “名人” 参加。若 “名人” 存在，请返回他/她的编号；若 “名人” 不存在，请返回 -1。

 

示例 1:

输入: graph = [
  [1,1,0],
  [0,1,0],
  [1,1,1]
]
输出: 1
解释: 有编号分别为 0、1 和 2 的三个人。graph[i][j] = 1 代表编号为 i 的人认识编号为 j 的人，而 graph[i][j] = 0 则代表编号为 i 的人不认识编号为 j 的人。“名人” 是编号 1 的人，因为 0 和 2 均认识他/她，但 1 不认识任何人。

示例 2:

输入: graph = [
  [1,0,1],
  [1,1,0],
  [0,1,1]
]
输出: -1
解释: 没有 “名人”

 

提示：

    n == graph.length
    n == graph[i].length
    2 <= n <= 100
    graph[i][j] 是 0 或 1.
    graph[i][i] == 1

 

进阶：如果允许调用 API knows 的最大次数为 3 * n ，你可以设计一个不超过最大调用次数的解决方案吗？

# The knows API is already defined for you.
# return a bool, whether a knows b
# def knows(a: int, b: int) -> bool:

class Solution:
    def findCelebrity(self, n: int) -> int:

class Solution(object):
    def findCelebrity(self, n):
        """
        :type n: int
        :rtype: int
        """
        # 比如有10个人 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        # 假设第‘0’个是名人
        candidate = 0;
        
        # 只要他认识除他之外的任何一个人，那么他一定不是名人，并把假设名人换成他认识的那个人
        # 如：‘0’ 认识 ‘3’，‘3’ 认识 ‘6’，但 ‘6’ 不认识他后面的所有人，那么此时 candidate = 6
        for i in range(1, n):
            if (knows(candidate, i)):
                candidate = i
        
        # candidate 一定不认识他之后的人
        # 所以只要判断：
        # 1. ‘0’ ~ ‘5’ 都认识 ‘6’ 且 ‘6’ 不认识他们任何一个
        for i in range(candidate):
            if (knows(candidate, i) or (not knows(i, candidate))):
                return -1
        # 2. ‘7’ ~ ‘9’ 都认识 ‘6’
        for i in range(candidate + 1, n):
            if not knows(i, candidate):
                return -1
            
        return candidate


class Solution(object):
    def findCelebrity(self, n):
        # 逻辑模拟
        # 假设名人为X，逐一判断X是否认识X+1、X+2...：
        # （1）如果X认识X+1，则X必然不是名人，X+1可能是名人，继续判断X+1是否认识X+2、X+3...
        # （2）如果X不认识X+1，则X+1必然不是名人，X可能是名人，继续判断X是否认识X+2、X+3...
        # 那么，通过用上述方式，用一次遍历找到X，满足：X被X-1认识，且X不认识后面的所有人。
        # 此时，再判断X是否同时满足：X不认识其他任何人，且其他任何人都认识X。
        # 如果满足条件，说明X为名人；否则说明不存在名人。
        X = 0 # 假设0是名人，开始逐一询问
        # 如果x认识i，则x必然不是名人，而i可能为名人，将询问对象改为i
        for i in range(n):
            if knows(X, i):
                X = i
        # 判断是否满足：X不认识其他任何人，且其他任何人都认识X
        for i in range(n):
            if i != X and ( knows(X, i) or not knows(i, X) ):
                return -1
        return X

545. 二叉树的边界

二叉树的 边界 是由 根节点 、左边界 、按从左到右顺序的 叶节点 和 逆序的右边界 ，按顺序依次连接组成。

左边界 是满足下述定义的节点集合：

    根节点的左子节点在左边界中。如果根节点不含左子节点，那么左边界就为 空 。
    如果一个节点在左边界中，并且该节点有左子节点，那么它的左子节点也在左边界中。
    如果一个节点在左边界中，并且该节点 不含 左子节点，那么它的右子节点就在左边界中。
    最左侧的叶节点 不在 左边界中。

右边界 定义方式与 左边界 相同，只是将左替换成右。即，右边界是根节点右子树的右侧部分；叶节点 不是 右边界的组成部分；如果根节点不含右子节点，那么右边界为 空 。

叶节点 是没有任何子节点的节点。对于此问题，根节点 不是 叶节点。

给你一棵二叉树的根节点 root ，按顺序返回组成二叉树 边界 的这些值。

 

示例 1：

输入：root = [1,null,2,3,4]
输出：[1,3,4,2]
解释：
- 左边界为空，因为二叉树不含左子节点。
- 右边界是 [2] 。从根节点的右子节点开始的路径为 2 -> 4 ，但 4 是叶节点，所以右边界只有 2 。
- 叶节点从左到右是 [3,4] 。
按题目要求依序连接得到结果 [1] + [] + [3,4] + [2] = [1,3,4,2] 。

示例 2：

输入：root = [1,2,3,4,5,6,null,null,null,7,8,9,10]
输出：[1,2,4,7,8,9,10,6,3]
解释：
- 左边界为 [2] 。从根节点的左子节点开始的路径为 2 -> 4 ，但 4 是叶节点，所以左边界只有 2 。
- 右边界是 [3,6] ，逆序为 [6,3] 。从根节点的右子节点开始的路径为 3 -> 6 -> 10 ，但 10 是叶节点。
- 叶节点从左到右是 [4,7,8,9,10]
按题目要求依序连接得到结果 [1] + [2] + [4,7,8,9,10] + [6,3] = [1,2,4,7,8,9,10,6,3] 。

 

提示：

    树中节点的数目在范围 [1, 104] 内
    -1000 <= Node.val <= 1000



python 遍历

分开遍历。注意排除根节点也是叶节点的情况。

def boundaryOfBinaryTree(self, root: TreeNode) -> List[int]:
    left, p = [], root.left
    while p:
        if p.left or p.right:
            left.append(p.val)
        p = p.left or p.right
    right, p = [], root.right
    while p:
        if p.left or p.right:
            right.append(p.val)
        p = p.right or p.left
    leaf, stack = [], [root]
    while stack:
        node = stack.pop()
        if not node.left and not node.right and node != root:
            leaf.append(node.val)
        stack.extend([child for child in [node.right, node.left] if child])
    return [root.val]+left+leaf+right[::-1]


分为 left leaf right 三个部分分别求解

需要注意的是，注意根节点的左右子树是否为空的情况

class Solution:
    def boundaryOfBinaryTree(self, root: TreeNode) -> List[int]:
        if not root: return []
        if not root.left and not root.right: return [root.val]
        # left
        left,right = [root.val], []
        node = root.left
        while node:
            left.append(node.val)
            if node.left:
                node= node.left
            else:
                node = node.right
        # right
        node = root.right
        while node:
            right.append(node.val)
            if node.right:
                node = node.right
            else:
                node = node.left
        # leaf
        self.leaf = []
        def dfs(root):
            if not root.left and not root.right:
                self.leaf.append(root.val)
            if root.left: dfs(root.left)
            if root.right: dfs(root.right)
        dfs(root)
        
        if len(left) != 1: # not root.left
            left = left[:-1]
        if len(right) != 0: # not root.right
            right = right[:-1]
            right = right[::-1]
        return left + self.leaf + right



285. 二叉搜索树中的中序后继

给定一棵二叉搜索树和其中的一个节点 p ，找到该节点在树中的中序后继。如果节点没有中序后继，请返回 null 。

节点 p 的后继是值比 p.val 大的节点中键值最小的节点。

 

示例 1：

输入：root = [2,1,3], p = 1
输出：2
解释：这里 1 的中序后继是 2。请注意 p 和返回值都应是 TreeNode 类型。

示例 2：

输入：root = [5,3,6,2,4,null,null,1], p = 6
输出：null
解释：因为给出的节点没有中序后继，所以答案就返回 null 了。

 

提示：

    树中节点的数目在范围 [1, 104] 内。
    -105 <= Node.val <= 105
    树中各节点的值均保证唯一。



# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution:
    def inorderSuccessor(self, root: 'TreeNode', p: 'TreeNode') -> 'TreeNode':
        if root is None:
            return None
        successor = None
        while root: # 进行二分查找
            if root.val < p.val: # p 在右边,抛弃左边
                root = root.right
            elif root.val > p.val: # p 在左边,抛弃右边,记得更新successor,因为左根右,根是左的后继
                successor = root
                root = root.left
            else: # 已经搜索到 p 了,接下来返回他的后继，有右孩子返回右子树最小(左根右,作为根,后继是右边最小)
                if root.right:
                    successor =  self.find_min(root.right)
                break
        return successor

    def find_min(sef, root):
        if root is None: # 后继无人的情况
            return None
        while root.left:
            root = root.left
        return root

如果自己手写BST类的话，一般也会写地板函数/天花板函数。

class Solution:
    def inorderSuccessor(self, root: 'TreeNode', p: 'TreeNode') -> 'TreeNode':
        if not root:
            return None
        if root.val <= p.val:
            return self.inorderSuccessor(root.right, p)
        temp = self.inorderSuccessor(root.left, p)
        if temp:
            return temp
        return root



186. 反转字符串中的单词 II

给你一个字符数组 s ，反转其中 单词 的顺序。

单词 的定义为：单词是一个由非空格字符组成的序列。s 中的单词将会由单个空格分隔。

必须设计并实现 原地 解法来解决此问题，即不分配额外的空间。

 

示例 1：

输入：s = ["t","h","e"," ","s","k","y"," ","i","s"," ","b","l","u","e"]
输出：["b","l","u","e"," ","i","s"," ","s","k","y"," ","t","h","e"]

示例 2：

输入：s = ["a"]
输出：["a"]

 

提示：

    1 <= s.length <= 105
    s[i] 可以是一个英文字母（大写或小写）、数字、或是空格 ' ' 。
    s 中至少存在一个单词
    s 不含前导或尾随空格
    题目数据保证：s 中的每个单词都由单个空格分隔



s[:] = list(' '.join(reversed(''.join(s).split())))


思路：
单词以空格分隔，先对每个单词进行reverse，再对整个字符串reverse，即可得到翻转单词顺序，但是单词中字母顺序不变的效果

class Solution:
    def reverseWords(self, s: List[str]) -> None:
        """
        Do not return anything, modify s in-place instead.
        """
        def Swap(s, left, right):
            while left < right:
                temp = s[left];
                s[left] = s[right]
                s[right] = temp
                left += 1
                right -= 1
        left = 0
        right = 0
        while right < len(s):
            if s[right] == ' ':
                Swap(s, left, right - 1);
                right += 1
                left = right
            else:
                right += 1
        Swap(s, left, len(s) - 1)
        Swap(s, 0, len(s) - 1)

作者：Duck-Bobby
链接：https://leetcode.cn/problems/reverse-words-in-a-string-ii/solution/c-java-shuang-zhi-zhen-by-sparkk-unnm/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

642. 设计搜索自动补全系统

为搜索引擎设计一个搜索自动补全系统。用户会输入一条语句（最少包含一个字母，以特殊字符 '#' 结尾）。

给定一个字符串数组 sentences 和一个整数数组 times ，长度都为 n ，其中 sentences[i] 是之前输入的句子， times[i] 是该句子输入的相应次数。对于除 ‘#’ 以外的每个输入字符，返回前 3 个历史热门句子，这些句子的前缀与已经输入的句子的部分相同。

下面是详细规则：

    一条句子的热度定义为历史上用户输入这个句子的总次数。
    返回前 3 的句子需要按照热度从高到低排序（第一个是最热门的）。如果有多条热度相同的句子，请按照 ASCII 码的顺序输出（ASCII 码越小排名越前）。
    如果满足条件的句子个数少于 3 ，将它们全部输出。
    如果输入了特殊字符，意味着句子结束了，请返回一个空集合。

实现 AutocompleteSystem 类：

    AutocompleteSystem(String[] sentences, int[] times): 使用数组sentences 和 times 对对象进行初始化。
    List<String> input(char c) 表示用户输入了字符 c 。
        如果 c == '#' ，则返回空数组 [] ，并将输入的语句存储在系统中。
        返回前 3 个历史热门句子，这些句子的前缀与已经输入的句子的部分相同。如果少于 3 个匹配项，则全部返回。

 

示例 1：

输入
["AutocompleteSystem", "input", "input", "input", "input"]
[[["i love you", "island", "iroman", "i love leetcode"], [5, 3, 2, 2]], ["i"], [" "], ["a"], ["#"]]
输出
[null, ["i love you", "island", "i love leetcode"], ["i love you", "i love leetcode"], [], []]

解释
AutocompleteSystem obj = new AutocompleteSystem(["i love you", "island", "iroman", "i love leetcode"], [5, 3, 2, 2]);
obj.input("i"); // return ["i love you", "island", "i love leetcode"]. There are four sentences that have prefix "i". Among them, "ironman" and "i love leetcode" have same hot degree. Since ' ' has ASCII code 32 and 'r' has ASCII code 114, "i love leetcode" should be in front of "ironman". Also we only need to output top 3 hot sentences, so "ironman" will be ignored.
obj.input(" "); // return ["i love you", "i love leetcode"]. There are only two sentences that have prefix "i ".
obj.input("a"); // return []. There are no sentences that have prefix "i a".
obj.input("#"); // return []. The user finished the input, the sentence "i a" should be saved as a historical sentence in system. And the following input will be counted as a new search.

 

提示:

    n == sentences.length
    n == times.length
    1 <= n <= 100
    1 <= sentences[i].length <= 100
    1 <= times[i] <= 50
    c 是小写英文字母， '#', 或空格 ' '
    每个被测试的句子将是一个以字符 '#' 结尾的字符 c 序列。
    每个被测试的句子的长度范围为 [1,200] 
    每个输入句子中的单词用单个空格隔开。
    input 最多被调用 5000 次



Python 字典树，每个节点直接存储相对应前缀的string

class Node:
    
    def __init__(self, char):
        self.char = char
        self.children = {}
        self.strings = {}

    def addString(self, string, occurs):
        self.strings[string] = self.strings.get(string, 0) + occurs
    
    def addChild(self, child):
        self.children[child.char] = child
    
    def getChild(self, char):
        return self.children.get(char)
    
    def recommend(self):
        topStrings = sorted([(v, k) for k, v in self.strings.items()], key=lambda x: (-x[0], x[1]))
        return [item[1] for item in topStrings[:3]]

class AutocompleteSystem:

    def __init__(self, sentences: List[str], times: List[int]):
        self.roots = {}
        self.inputPreNode = None
        self.inputSentence = ""
        self.isStart = True

        n = len(sentences)
        for i in range(n):
            sentence = sentences[i]
            self.addSentence(sentence, times[i])
    
    def addSentence(self, sentence, time):
        pre = None
        for j in range(len(sentence)):
            c = sentence[j]
            # print(c)
            if j == 0:
                if c in self.roots:
                    cur = self.roots[c]
                else:
                    cur = Node(c)
                    self.roots[c] = cur
                cur.addString(sentence, time)
            else:
                cur = pre.getChild(c)
                if cur is None:
                    cur = Node(c)
                    pre.addChild(cur)
                cur.addString(sentence, time)
                
            pre = cur

    def input(self, c: str) -> List[str]:
        if c == "#":
            self.addSentence(self.inputSentence, 1)
            self.inputPreNode = None
            self.inputSentence = ""
            self.isStart = True
            return []
        else:
            self.inputSentence += c
            if self.inputPreNode is None:
                if self.isStart:
                    cur = self.roots.get(c)
                    self.isStart = False
                else:
                    return []
            else:
                # print(self.inputPreNode.children, c)
                cur = self.inputPreNode.getChild(c)
            
            self.inputPreNode = cur
            
            if cur is None:
                return []
            else:
                return cur.recommend()



利用字典树存储所有输入过的句子

维护当前节点self.curNode和当前输入的句子self.curWord，根据题意在输入字符过程中更新字典树即可

当需要返回热门前三的句子时，从当前节点开始做BFS，把接下来所有的句子和其对应次数加入到列表temp中，然后根据次数和ASCII码排序，将前三的句子和当前已经输入的句子拼接，即得到热门前三的句子

class AutocompleteSystem:

    def __init__(self, sentences: List[str], times: List[int]):
        self.words = {}
        self.curNode = self.words   #当前节点
        self.curWord = ''  #当前已输入的字符串
        for word, time in zip(sentences, times):  #先更新句子表里的句子和对应次数
            cur = self.words
            for n in word:
                if n not in cur:
                    cur[n] = {}
                cur = cur[n]
            cur['times'] = time

    def input(self, c: str) -> List[str]:
        if c == '#':
            if 'times' in self.curNode:
                self.curNode['times'] += 1
            else:
                self.curNode['times'] = 1
            self.curNode = self.words
            self.curWord = ''
            return []
        else:
            if c not in self.curNode:
                self.curNode[c] = {}
            self.curNode = self.curNode[c]
            self.curWord += c
            return self.bfs()
    
    def bfs(self):  #从当前节点开始，利用BFS遍历所有句子
        temp = []
        cur = self.curNode
        Q = deque([[cur, '']])
        while Q:
            cur, word = Q.popleft()
            if 'times' in cur:  #推入次数+当前单词
                temp.append([cur['times'], word])
            for n in cur.keys():
                if n != 'times':
                    Q.append([cur[n], word + n])
        temp.sort(key = lambda x: [-x[0], x[1]])  #首先根据次数排序，然后根据ASCII码排序
        ret = []
        for i in range(min(3, len(temp))):  #和当前已输入的句子拼接返回
            ret.append(self.curWord + temp[i][1])
        return ret


428. 序列化和反序列化 N 叉树

序列化是指将一个数据结构转化为位序列的过程，因此可以将其存储在文件中或内存缓冲区中，以便稍后在相同或不同的计算机环境中恢复结构。

设计一个序列化和反序列化 N 叉树的算法。一个 N 叉树是指每个节点都有不超过 N 个孩子节点的有根树。序列化 / 反序列化算法的算法实现没有限制。你只需要保证 N 叉树可以被序列化为一个字符串并且该字符串可以被反序列化成原树结构即可。

例如，你需要序列化下面的 3-叉 树。

 

 

为 [1 [3[5 6] 2 4]]。你不需要以这种形式完成，你可以自己创造和实现不同的方法。

或者，您可以遵循 LeetCode 的层序遍历序列化格式，其中每组孩子节点由空值分隔。

例如，上面的树可以序列化为 [1,null,2,3,4,5,null,null,6,7,null,8,null,9,10,null,null,11,null,12,null,13,null,null,14]

你不一定要遵循以上建议的格式，有很多不同的格式，所以请发挥创造力，想出不同的方法来完成本题。

 

示例 1:

输入: root = [1,null,2,3,4,5,null,null,6,7,null,8,null,9,10,null,null,11,null,12,null,13,null,null,14]
输出: [1,null,2,3,4,5,null,null,6,7,null,8,null,9,10,null,null,11,null,12,null,13,null,null,14]

示例 2:

输入: root = [1,null,3,2,4,null,5,6]
输出: [1,null,3,2,4,null,5,6]

示例 3:

输入: root = []
输出: []

 

提示：

    树中节点数目的范围是 [0, 104].
    0 <= Node.val <= 104
    N 叉树的高度小于等于 1000
    不要使用类成员 / 全局变量 / 静态变量来存储状态。你的序列化和反序列化算法应是无状态的。

"""
# Definition for a Node.
class Node(object):
    def __init__(self, val=None, children=[]):
        self.val = val
        self.children = children
"""

class Codec:
    def serialize(self, root: 'Node') -> str:
        """Encodes a tree to a single string.
        
        :type root: Node
        :rtype: str
        """
        
	
    def deserialize(self, data: str) -> 'Node':
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: Node
        """
        

# Your Codec object will be instantiated and called as such:
# codec = Codec()
# codec.deserialize(codec.serialize(root))

思路一：dfs

class Codec:

    def serialize(self, root: 'Node') -> str:
        """Encodes a tree to a single string.
        
        :type root: Node
        :rtype: str
        """
        res = []  

        def dfs(root):
            if not root: return 
            res.append(str(root.val))
            res.append(str(len(root.children)))
            for ch in root.children:
                dfs(ch)
        
        dfs(root)
        # print(res)
        return ",".join(res)

    def deserialize(self, data: str) -> 'Node':
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: Node
        """
        if not data: return 
        data = iter(data.split(","))
        
        def dfs():
            root = Node(int(next(data)), [])
            num = int(next(data))
            for _ in range(num):
                root.children.append(dfs())
            return root
        
        return dfs()

思路二：层序遍历

class Codec:

    def serialize(self, root: 'Node') -> str:
        """Encodes a tree to a single string.
        
        :type root: Node
        :rtype: str
        """
        from collections import deque
        if not root: return ""  
        queue = deque([root])
        res = []
        while queue:
            node = queue.pop()
            res.append(str(node.val))
            res.append(str(len(node.children)))
            for ch in node.children:
                queue.appendleft(ch)
        return ",".join(res)

    def deserialize(self, data: str) -> 'Node':
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: Node
        """
       #  print(data)
        from collections import deque
        if not data: return 
        data = iter(data.split(","))
        root = Node(int(next(data)), [])
        queue = deque([[root, int(next(data))]])

        while queue:
            node, num = queue.pop()
            for _ in range(num):
                tmp = int(next(data))
                tmp_num = int(next(data))
                tmp_node = Node(tmp, [])
                node.children.append(tmp_node)
                queue.appendleft([tmp_node, tmp_num])
        return root

LeetCode 类似题型:

297. 二叉树的序列化与反序列化

449. 序列化和反序列化二叉搜索树

428. 序列化和反序列化 N 叉树


JSON 槇鵏嚽！

class Codec:
    def serialize(self, root: 'Node') -> str:
        def dfs(root):
            if not root: return None
            temp = {"val": root.val, "kids": []}
            for kid in root.children:
                temp["kids"].append(dfs(kid))
            return temp
        return json.dumps(dfs(root))
	
    def deserialize(self, data: str) -> 'Node':
        def build(root):
            if not root: return None
            cur = Node(root["val"], [])
            for kid in root["kids"]:
                cur.children.append(build(kid))
            return cur
        return build(json.loads(data))



思路和心得：
(1)dfs是最简单的

"root.val 儿子个数 儿子们递归的结果"

解码的时候也按照这个顺序解码

c++借助stringstream， 注意要&

python3很方便，怎么写都行

"""
# Definition for a Node.
class Node(object):
    def __init__(self, val=None, children=None):
        self.val = val
        self.children = children
"""

class Codec:
    def serialize(self, root: 'Node') -> str:
        """Encodes a tree to a single string.
        
        :type root: Node
        :rtype: str
        """
        if root == None:
            return "#"
        data = ""
        data += str(root.val) + '-' + str(len(root.children))
        for child in root.children:
            data += '-' + self.serialize(child)
        return data        
	
    def deserialize(self, data: str) -> 'Node':
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: Node
        """
        if data == "#":
            return None
        data = data.split('-')
        return self.dfs(data)

    def dfs(self, data: List[str]) -> 'Node':
        #建root结点
        val = int(data.pop(0))
        root = Node(val)
        root.children = []
        #递归儿子们
        size = int(data.pop(0))
        for _ in range(size):
            root.children.append(self.dfs(data))
        return root
        

# Your Codec object will be instantiated and called as such:
# codec = Codec()
# codec.deserialize(codec.serialize(root))

(2)bfs

既是bfs

必用队列成，、必用队列解

"""
# Definition for a Node.
class Node(object):
    def __init__(self, val=None, children=None):
        self.val = val
        self.children = children
"""

class Codec:
    def serialize(self, root: 'Node') -> str:
        """Encodes a tree to a single string.
        
        :type root: Node
        :rtype: str
        """
        if root == None:
            return ""
        data = ""
        data += str(root.val) + '#'
        #结点#儿子1，儿子2，儿子3#”
        Q = collections.deque()
        Q.append(root)
        while Q:
            t = Q.popleft()
            for child in t.children:
                data += str(child.val) + ','
                Q.append(child)
            data += '#'
        return data
	
    def deserialize(self, data: str) -> 'Node':
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: Node
        """
        if data == "":
            return None
        dataLen = len(data)
        ID = 0
        val = 0
        while ID < dataLen and data[ID].isdigit():
            val = val * 10 + int(data[ID])
            ID += 1
        root = Node(val)
        ID += 1         #跳过'#'
        Q = collections.deque()
        Q.append(root)
        while Q and ID < dataLen:
            t = Q.popleft()
            t.children = []
            while ID < dataLen and data[ID] != '#':
                val = 0
                while ID < dataLen and data[ID].isdigit():
                    val = val * 10 + int(data[ID])
                    ID += 1
                child = Node(val)
                ID += 1         #跳过','
                t.children.append(child)
                Q.append(child) 
            ID += 1             #跳过'#'

        return root
# Your Codec object will be instantiated and called as such:
# codec = Codec()
# codec.deserialize(codec.serialize(root))



作者：code_learner
链接：https://leetcode.cn/problems/serialize-and-deserialize-n-ary-tree/solution/cpython3-1dfs-2bfs-by-hanxin_hanxin-606j/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

269. 火星词典

现有一种使用英语字母的火星语言，这门语言的字母顺序与英语顺序不同。

给你一个字符串列表 words ，作为这门语言的词典，words 中的字符串已经 按这门新语言的字母顺序进行了排序 。

请你根据该词典还原出此语言中已知的字母顺序，并 按字母递增顺序 排列。若不存在合法字母顺序，返回 "" 。若存在多种可能的合法字母顺序，返回其中 任意一种 顺序即可。

字符串 s 字典顺序小于 字符串 t 有两种情况：

    在第一个不同字母处，如果 s 中的字母在这门外星语言的字母顺序中位于 t 中字母之前，那么 s 的字典顺序小于 t 。
    如果前面 min(s.length, t.length) 字母都相同，那么 s.length < t.length 时，s 的字典顺序也小于 t 。

 

示例 1：

输入：words = ["wrt","wrf","er","ett","rftt"]
输出："wertf"

示例 2：

输入：words = ["z","x"]
输出："zx"

示例 3：

输入：words = ["z","x","z"]
输出：""
解释：不存在合法字母顺序，因此返回 "" 。

 

提示：

    1 <= words.length <= 100
    1 <= words[i].length <= 100
    words[i] 仅由小写英文字母组成



class Solution:
    def alienOrder(self, words: List[str]) -> str:
        # construct the adjacency lists based on words
        adjacency = {c:set() for word in words for c in word}
        # 遍历外星文
        for i in range(len(words)-1):
            word1,word2 = words[i],words[i+1]
            minLen = min(len(word1),len(word2))
            if word1[:minLen] == word2[:minLen] and len(word1)>len(word2):
                # return "" as the base case
                return ""
            for j in range(minLen):
                if word1[j] != word2[j]:
                    adjacency[word1[j]].add(word2[j])
                    break
        
        # start dfs algorithm to traverse the graph
        visited = {}
        # initialize a stack
        stack = []

        def dfs(cur):
            if cur in visited: return visited[cur]
            
            visited[cur] = True
            for neighbor in adjacency[cur]:
                if dfs(neighbor): return True
            visited[cur] = False
            stack.append(cur)
        
        for letter in adjacency:
            if dfs(letter):
                return ""
        return "".join(stack[::-1])

前言

这道题是拓扑排序问题。火星词典中的字母和字母顺序可以看成有向图，字典顺序即为所有字母的一种排列，满足每一条有向边的起点字母和终点字母的顺序都和这两个字母在排列中的顺序相同，该排列即为有向图的拓扑排序。

只有当有向图中无环时，才有拓扑排序，且拓扑排序可能不止一种。如果有向图中有环，则环内的字母不存在符合要求的排列，因此没有拓扑排序。

使用拓扑排序求解时，将火星词典中的每个字母看成一个节点，将字母之间的顺序关系看成有向边。对于火星词典中的两个相邻单词，同时从左到右遍历，当遇到第一个不相同的字母时，该位置的两个字母之间即存在顺序关系。

以下两种情况不存在合法字母顺序：

    字母之间的顺序关系存在由至少 222 个字母组成的环，例如 words=[“a",“b",“a"]\textit{words} = [\text{``a"}, \text{``b"}, \text{``a"}]words=[“a",“b",“a"]；

    相邻两个单词满足后面的单词是前面的单词的前缀，且后面的单词的长度小于前面的单词的长度，例如 words=[“ab",“a"]\textit{words} = [\text{``ab"}, \text{``a"}]words=[“ab",“a"]。

其余情况下都存在合法字母顺序，可以使用拓扑排序得到字典顺序。

拓扑排序可以使用深度优先搜索或广度优先搜索实现，以下分别介绍两种实现方法。
方法一：拓扑排序 + 深度优先搜索

使用深度优先搜索实现拓扑排序的总体思想是：对于一个特定节点，如果该节点的所有相邻节点都已经搜索完成，则该节点也会变成已经搜索完成的节点，在拓扑排序中，该节点位于其所有相邻节点的前面。一个节点的相邻节点指的是从该节点出发通过一条有向边可以到达的节点。

由于拓扑排序的顺序和搜索完成的顺序相反，因此需要使用一个栈存储所有已经搜索完成的节点。深度优先搜索的过程中需要维护每个节点的状态，每个节点的状态可能有三种情况：「未访问」、「访问中」和「已访问」。初始时，所有节点的状态都是「未访问」。

每一轮搜索时，任意选取一个「未访问」的节点 uuu，从节点 uuu 开始深度优先搜索。将节点 uuu 的状态更新为「访问中」，对于每个与节点 uuu 相邻的节点 vvv，判断节点 vvv 的状态，执行如下操作：

    如果节点 vvv 的状态是「未访问」，则继续搜索节点 vvv；

    如果节点 vvv 的状态是「访问中」，则找到有向图中的环，因此不存在拓扑排序；

    如果节点 vvv 的状态是「已访问」，则节点 vvv 已经搜索完成并入栈，节点 uuu 尚未入栈，因此节点 uuu 的拓扑顺序一定在节点 vvv 的前面，不需要执行任何操作。

当节点 uuu 的所有相邻节点的状态都是「已访问」时，将节点 uuu 的状态更新为「已访问」，并将节点 uuu 入栈。

当所有节点都访问结束之后，如果没有找到有向图中的环，则存在拓扑排序，所有节点从栈顶到栈底的顺序即为拓扑排序。

实现方面，由于每个节点是一个字母，因此可以使用字符数组代替栈，当节点入栈时，在字符数组中按照从后往前的顺序依次填入每个字母。当所有节点都访问结束之后，将字符数组转成字符串，即为字典顺序。

class Solution:
    def alienOrder(self, words: List[str]) -> str:
        g = {}
        for c in words[0]:
            g[c] = []
        for s, t in pairwise(words):
            for c in t:
                g.setdefault(c, [])
            for u, v in zip(s, t):
                if u != v:
                    g[u].append(v)
                    break
            else:
                if len(s) > len(t):
                    return ""

        VISITING, VISITED = 1, 2
        states = {}
        order = []
        def dfs(u: str) -> bool:
            states[u] = VISITING
            for v in g[u]:
                if v not in states:
                    if not dfs(v):
                        return False
                elif states[v] == VISITING:
                    return False
            order.append(u)
            states[u] = VISITED
            return True

        return ''.join(reversed(order)) if all(dfs(u) for u in g if u not in states) else ""

复杂度分析

    时间复杂度：O(n×L+∣Σ∣)O(n \times L + |\Sigma|)O(n×L+∣Σ∣)，其中 nnn 是数组 words\textit{words}words 的长度，即字典中的单词数，LLL 是字典中的平均单词长度，Σ\SigmaΣ 是字典中的字母集合。遍历字典构造有向图需要 O(n×L)O(n \times L)O(n×L) 的时间，由于有向图包含最多 n−1n - 1n−1 条边和 ∣Σ∣|\Sigma|∣Σ∣ 个节点，因此深度优先搜索需要 O(n+∣Σ∣)O(n + |\Sigma|)O(n+∣Σ∣) 的时间，总时间复杂度是 O(n×L+n+∣Σ∣)=O(n×L+∣Σ∣)O(n \times L + n + |\Sigma|) = O(n \times L + |\Sigma|)O(n×L+n+∣Σ∣)=O(n×L+∣Σ∣)。

    空间复杂度：O(n+∣Σ∣)O(n + |\Sigma|)O(n+∣Σ∣)，其中 nnn 是数组 words\textit{words}words 的长度，即字典中的单词数，Σ\SigmaΣ 是字典中的字母集合。空间复杂度主要取决于存储有向图需要的空间，有向图包含最多 n−1n - 1n−1 条边和 ∣Σ∣|\Sigma|∣Σ∣ 个节点。

方法二：拓扑排序 + 广度优先搜索

方法一使用深度优先搜索实现拓扑排序，根据每个节点搜索完成的顺序反向得到拓扑排序。使用广度优先搜索实现拓扑排序，则可以正向得到拓扑排序。

首先计算每个节点的入度，只有入度为 000 的节点可能是拓扑排序中最前面的节点。当一个节点加入拓扑排序之后，该节点的所有相邻节点的入度都减 111，表示相邻节点少了一条入边。当一个节点的入度变成 000，则该节点前面的节点都已经加入拓扑排序，该节点也可以加入拓扑排序。

具体做法是，使用队列存储可以加入拓扑排序的节点，初始时将所有入度为 000 的节点入队列。每次将一个节点出队列并加入拓扑排序中，然后将该节点的所有相邻节点的入度都减 111，如果一个相邻节点的入度变成 000，则将该相邻节点入队列。重复上述操作，直到队列为空时，广度优先搜索结束。

如果有向图中无环，则每个节点都将加入拓扑排序，因此拓扑排序的长度等于字典中的字母个数。如果有向图中有环，则环中的节点不会加入拓扑排序，因此拓扑排序的长度小于字典中的字母个数。广度优先搜索结束时，判断拓扑排序的长度是否等于字典中的字母个数，即可判断有向图中是否有环。

    如果拓扑排序的长度等于字典中的字母个数，则拓扑排序包含字典中的所有字母，返回拓扑排序；

    如果拓扑排序的长度小于字典中的字母个数，则有向图中有环，不存在拓扑排序。

class Solution:
    def alienOrder(self, words: List[str]) -> str:
        g = defaultdict(list)
        inDeg = {c: 0 for c in words[0]}
        for s, t in pairwise(words):
            for c in t:
                inDeg.setdefault(c, 0)
            for u, v in zip(s, t):
                if u != v:
                    g[u].append(v)
                    inDeg[v] += 1
                    break
            else:
                if len(s) > len(t):
                    return ""

        q = [u for u, d in inDeg.items() if d == 0]
        for u in q:
            for v in g[u]:
                inDeg[v] -= 1
                if inDeg[v] == 0:
                    q.append(v)
        return ''.join(q) if len(q) == len(inDeg) else ""

复杂度分析

    时间复杂度：O(n×L+∣Σ∣)O(n \times L + |\Sigma|)O(n×L+∣Σ∣)，其中 nnn 是数组 words\textit{words}words 的长度，即字典中的单词数，LLL 是字典中的平均单词长度，Σ\SigmaΣ 是字典中的字母集合。遍历字典构造有向图需要 O(n×L)O(n \times L)O(n×L) 的时间，由于有向图包含最多 n−1n - 1n−1 条边和 ∣Σ∣|\Sigma|∣Σ∣ 个节点，因此广度优先搜索需要 O(n+∣Σ∣)O(n + |\Sigma|)O(n+∣Σ∣) 的时间，总时间复杂度是 O(n×L+n+∣Σ∣)=O(n×L+∣Σ∣)O(n \times L + n + |\Sigma|) = O(n \times L + |\Sigma|)O(n×L+n+∣Σ∣)=O(n×L+∣Σ∣)。

    空间复杂度：O(n+∣Σ∣)O(n + |\Sigma|)O(n+∣Σ∣)，其中 nnn 是数组 words\textit{words}words 的长度，即字典中的单词数，Σ\SigmaΣ 是字典中的字母集合。空间复杂度主要取决于存储有向图需要的空间，有向图包含最多 n−1n - 1n−1 条边和 ∣Σ∣|\Sigma|∣Σ∣ 个节点。

作者：LeetCode-Solution
链接：https://leetcode.cn/problems/alien-dictionary/solution/huo-xing-ci-dian-by-leetcode-solution-nr0l/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

 解题思路

    遍历words统计所有出现的字母
    遍历words中相邻的每对儿，统计这两对儿字典序不同的原因（第一个出现不同的地方，如果都一样且前面的长，可以直接返回）
    从所有没有入度的字母开始，拓扑遍历所有转移（因为转移是满足字典序的，入度为0时加入的话，所有字典序比它小的都已经加入了，可以加入）

PS:
WA了三发才理解题意，
所有字母都要出现在最终的答案中，不是只有转移的。
别的就没啥了。
代码

class Solution:
    def alienOrder(self, words: List[str]) -> str:
        graph, s = defaultdict(list), set()
        for w in words:
            s = s.union(set(w))
        d = [0] * 26
        for a, b in pairwise(words):
            for ca, cb in zip(a, b):
                if ca != cb:
                    graph[ca].append(cb)
                    d[ord(cb) - ord('a')] += 1
                    break
            else: 
                if len(a) > len(b):
                    return ""
        start = [k for k in s if d[ord(k) - ord('a')] == 0]
        for ch in start:
            for nxt in graph[ch]:
                d[v := ord(nxt) - ord('a')] -= 1
                if not d[v]:
                    start.append(nxt)
        return "".join(start) if len(start) == len(s) else ""

作者：himymBen
链接：https://leetcode.cn/problems/alien-dictionary/solution/by-himymben-jall/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

362. 敲击计数器

设计一个敲击计数器，使它可以统计在过去 5 分钟内被敲击次数。（即过去 300 秒）

您的系统应该接受一个时间戳参数 timestamp (单位为 秒 )，并且您可以假定对系统的调用是按时间顺序进行的(即 timestamp 是单调递增的)。几次撞击可能同时发生。

实现 HitCounter 类:

    HitCounter() 初始化命中计数器系统。
    void hit(int timestamp) 记录在 timestamp ( 单位为秒 )发生的一次命中。在同一个 timestamp 中可能会出现几个点击。
    int getHits(int timestamp) 返回 timestamp 在过去 5 分钟内(即过去 300 秒)的命中次数。

示例 1:

输入：
["HitCounter", "hit", "hit", "hit", "getHits", "hit", "getHits", "getHits"]
[[], [1], [2], [3], [4], [300], [300], [301]]
输出：
[null, null, null, null, 3, null, 4, 3]

解释：
HitCounter counter = new HitCounter();
counter.hit(1);// 在时刻 1 敲击一次。
counter.hit(2);// 在时刻 2 敲击一次。
counter.hit(3);// 在时刻 3 敲击一次。
counter.getHits(4);// 在时刻 4 统计过去 5 分钟内的敲击次数, 函数返回 3 。
counter.hit(300);// 在时刻 300 敲击一次。
counter.getHits(300); // 在时刻 300 统计过去 5 分钟内的敲击次数，函数返回 4 。
counter.getHits(301); // 在时刻 301 统计过去 5 分钟内的敲击次数，函数返回 3 。

 

提示:

    1 <= timestamp <= 2 * 109
    所有对系统的调用都是按时间顺序进行的（即 timestamp 是单调递增的）
    hit and getHits 最多被调用 300 次

 

进阶: 如果每秒的敲击次数是一个很大的数字，你的计数器可以应对吗？

来源：力扣（LeetCode）
链接：https://leetcode.cn/problems/design-hit-counter
著作权归领扣网络所有。商业转载请联系官方授权，非商业转载请注明出处。

两种写法:

第一种

from collections import deque


class HitCounter:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.queue = deque()

    def hit(self, timestamp: int) -> None:
        """
        Record a hit.
        @param timestamp - The current timestamp (in seconds granularity).
        """
        self.queue.appendleft(timestamp)

    def getHits(self, timestamp: int) -> int:
        """
        Return the number of hits in the past 5 minutes.
        @param timestamp - The current timestamp (in seconds granularity).
        """
        while self.queue and timestamp - self.queue[-1] + 1 > 300:
            self.queue.pop()
        return len(self.queue)

第二种

from collections import deque, defaultdict


class HitCounter:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.time = deque()
        self.lookup = defaultdict(int)
        self.now = 0

    def hit(self, timestamp: int) -> None:
        """
        Record a hit.
        @param timestamp - The current timestamp (in seconds granularity).
        """
        # print(self.lookup, self.time)
        while self.time and timestamp - self.time[-1] + 1 > 300:
            self.lookup.pop(self.time.pop())
        if timestamp > self.now:
            self.time.appendleft(timestamp)
            self.now = timestamp
        self.lookup[timestamp] += 1

    def getHits(self, timestamp: int) -> int:
        """
        Return the number of hits in the past 5 minutes.
        @param timestamp - The current timestamp (in seconds granularity).
        """
        while self.time and timestamp - self.time[-1] + 1 > 300:
            self.lookup.pop(self.time.pop())
        return sum(self.lookup.values())
        



看到时间单调递增，我们就想到可以使用二分查找来解决。具体代码如下。

class HitCounter:

    def __init__(self):
        self.sum_arr = []
        self.timestamp_arr = []

    def hit(self, timestamp: int) -> None:
        if self.timestamp_arr and timestamp == self.timestamp_arr[-1]:
            self.sum_arr[-1] += 1
        else:
            self.timestamp_arr.append(timestamp)
            if self.sum_arr:
                self.sum_arr.append(self.sum_arr[-1] + 1)
            else:
                self.sum_arr.append(1)

    def getHits(self, timestamp: int) -> int:
        if not self.timestamp_arr: return 0
        
        start = timestamp - 300
        # 二分查找，找到开始时刻的数组下标。小于等于 start 时刻的最右边。
        l, r = 0, len(self.timestamp_arr) - 1

        while l < r:
            mid = l + r + 1 >> 1
            if self.timestamp_arr[mid] <= start:
                l = mid
            else:
                r = mid - 1
        
        if self.timestamp_arr[l] > start:
            a = 0
        else:
            a = self.sum_arr[l]

        # 二分查找，找到当前时刻的数组下标。小于等于 timestamp 的最右边。
        l, r = 0, len(self.timestamp_arr) - 1
        while l < r:
            mid = l + r + 1 >> 1
            if self.timestamp_arr[mid] <= timestamp:
                l = mid
            else:
                r = mid - 1
        
        if self.timestamp_arr[l] > timestamp:
            b = 0
        else:
            b = self.sum_arr[l]

        # 做差得解。
        return b - a
        


# Your HitCounter object will be instantiated and called as such:
# obj = HitCounter()
# obj.hit(timestamp)
# param_2 = obj.getHits(timestamp)

作者：be_a_better_coder
链接：https://leetcode.cn/problems/design-hit-counter/solution/er-fen-cha-zhao-by-be_a_better_coder-l6ca/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

588. 设计内存文件系统

设计一个内存文件系统，模拟以下功能：

实现文件系统类:

    FileSystem() 初始化系统对象
    List<String> ls(String path)
        如果 path 是一个文件路径，则返回一个仅包含该文件名称的列表。
        如果 path 是一个目录路径，则返回该目录中文件和 目录名 的列表。

          答案应该 按字典顺序 排列。

    void mkdir(String path) 根据给定的路径创建一个新目录。给定的目录路径不存在。如果路径中的中间目录不存在，您也应该创建它们。
    void addContentToFile(String filePath, String content)
        如果 filePath 不存在，则创建包含给定内容 content的文件。
        如果 filePath 已经存在，将给定的内容 content附加到原始内容。
    String readContentFromFile(String filePath) 返回 filePath下的文件内容。

 

示例 1:

输入: 
["FileSystem","ls","mkdir","addContentToFile","ls","readContentFromFile"]
[[],["/"],["/a/b/c"],["/a/b/c/d","hello"],["/"],["/a/b/c/d"]]
输出:
[null,[],null,null,["a"],"hello"]

解释:
FileSystem fileSystem = new FileSystem();
fileSystem.ls("/");                         // 返回 []
fileSystem.mkdir("/a/b/c");
fileSystem.addContentToFile("/a/b/c/d", "hello");
fileSystem.ls("/");                         // 返回 ["a"]
fileSystem.readContentFromFile("/a/b/c/d"); // 返回 "hello"

 

注意:

    1 <= path.length, filePath.length <= 100
    path 和 filePath 都是绝对路径，除非是根目录 ‘/’ 自身，其他路径都是以 ‘/’ 开头且 不 以 ‘/’ 结束。
    你可以假定所有操作的参数都是有效的，即用户不会获取不存在文件的内容，或者获取不存在文件夹和文件的列表。
    你可以假定所有文件夹名字和文件名字都只包含小写字母，且同一文件夹下不会有相同名字的文件夹或文件。
    1 <= content.length <= 50
    ls, mkdir, addContentToFile, and readContentFromFile 最多被调用 300 次



class FileSystemItem:
    def __init__(self):
        self.isfile = False
        self.items = {}
        self.content = ""

class FileSystem:
    # 文件和目录使用同一份Dict，方便遍历文件夹下所有子元素. 使用这种维护文件结构的优势是很容易添加更多指令。重命名文件或者文件夹非常容易，因为我们只需要以新名字创建一个新的文件夹结构或者文件并删除原本的条目即可。从一个路径移动子文件夹到另一个路径也很容易，因为我们需要做的只是获得相应子文件夹类的地址，然后在新文件路径下赋新的值。如果文件夹数目非常大，我们会因为isfile和content浪费多余的空间，这部分空间在方法1中不会存在。这个方法的一个问题是如果我们只想要给定路径中文件夹的列表，而不是文件的列表，访问将会变得低效。我们需要遍历当前文件夹所有内容一遍并检查是否是文件夹，才能得到我们想要的数据。
    def __init__(self):
        # 现在文件夹数据结构只有一个统一的files哈希表，它保存了当前路径下所有的文件和子文件夹。除此以外，每个条目都有一个变量isfile，如果为True表示当前是一个文件，否则是一个文件夹。进一步的，因为我们将文件夹和文件统一保存，我们还需要一个content的条目，如果isfile为True那么它保存了当前文件的内容。对于子文件夹, content为空
        self.root = FileSystemItem()

    # 我们将输入文件夹路径以'/'为依据划分开并得到每一层文件夹名字的数组path_list。然后我们遍历文件树并更新指针让其指向当前的文件所代表的树节点。最后，我们要么停在最后一个文件夹里，或者停在一个文件上。如果输入最后一级是文件名字，我们只需要返回这个文件名。如果最后一级是文件夹，我们从它的items哈希表中得到所有子文件夹的名字, 排序后返回. ls的时间复杂度是O(m+n+klog(k))。这里m是输入字符串的长度，我们需要扫描输入字符串一次并获得每一层的文件名。n是最后一层文件夹的深度，我们需要进入n层的文件树以到达最后一层文件所在路径。k是最后一层的文件和文件夹总数目. 
    def ls(self, path: str) -> List[str]:
        item_object = self.root
        files = []
        if path != "/":
            path_list = path.split("/")
            for i in range(1, len(path_list)):
                item_object = item_object.items[path_list[i]]
            # 最后，我们要么停在最后一个文件夹里，或者停在一个文件上。
            if item_object.isfile:
                # 如果输入最后一级是文件名字，我们只需要返回这个文件名
                files.append(path_list[-1])
                return files
        # 如果最后一级是文件夹，我们从它的items哈希表中得到所有子文件夹的名字。
        files = list(item_object.items.keys())
        files.sort()
        return files

    # 如ls, 我们从根文件夹开始逐层遍历文件树。一旦我们在mkdir中遇到一个不存在的文件夹,我们在最后一个文件夹的items结构中创建一个新的条目并把它的子文件夹列表初始化为空列表。我们持续这一过程直到我们到达最后一级目录。时间 O(m+n),n是最后一层文件夹的深度。空间为O(1)
    def mkdir(self, path: str) -> None:
        item_object = self.root
        path_list = path.split("/")
        for i in range(1, len(path_list)):
            # 一旦我们在mkdir中遇到一个不存在的文件夹,我们在最后一个文件夹的items结构中创建一个新的条目并把它的子文件夹列表初始化为空列表。
            if path_list[i] not in item_object.items:
                item_object.items[path_list[i]] = FileSystemItem()
            # 我们持续这一过程直到我们到达最后一级目录
            item_object = item_object.items[path_list[i]]

    # 如ls, 我们逐层遍历文件路径。当我们到达文件所在文件夹时，我们检查items中是否已经存在文件。如果文件已经存在，我们把当前内容添加到该文件的末尾，在对应文件的content部分添加。如果文件不存在，我们在当前文件夹的items中添加一个新的条目并将它的内容初始化为当前内容。时间 O(m+n),n是最后一层文件夹的深度。空间为O(1)
    def addContentToFile(self, filePath: str, content: str) -> None:
        item_object = self.root
        path_list = filePath.split("/")
        for i in range(1, len(path_list) - 1):
            # 逐层遍历文件路径
            item_object = item_object.items[path_list[i]]
        
        if path_list[-1] not in item_object.items:
            # 如果文件不存在，我们在当前文件夹的items中添加一个新的条目并将它的内容初始化为当前内容
            item_object.items[path_list[-1]] = FileSystemItem()
        file_object = item_object.items[path_list[-1]]

        # 如果文件已经存在，我们把当前内容添加到该文件的末尾，在对应文件的content部分添加
        file_object.isfile = True
        file_object.content += content

    # 我们通过逐层遍历到达最后一层文件夹目录。在最后一层文件夹中，我们在items中找到文件名的条目，然后返回它对应的文件内容。时间 O(m+n),n是最后一层文件夹的深度。空间为O(1) 
    def readContentFromFile(self, filePath: str) -> str:
        item_object = self.root
        path_list = filePath.split("/")
        for i in range(1, len(path_list) - 1):
            # 逐层遍历文件路径
            item_object = item_object.items[path_list[i]]
        # # 在最后一层文件夹中，找到文件名的条目, 返回它对应的文件内容
        file_object = item_object.items[path_list[-1]]
        return file_object.content

class Dir:
    def __init__(self):
        self.dirs = {}
        self.files = {}

class FileSystem:
    # 文件和目录分别使用不同的2份Dict，方便独立处理文件元素以及文件夹元素. 优势是扩展性很好，可以很方便地增加更多指令.重命名文件夹和文件也非常容易，因为我们需要做的只是用新的名字创建一个新的文件夹结构或者文件并删除原本的条目即可。将一个子文件夹从一个目录移到另一个目录页非常容易，因为我们只需要获取相应的子文件夹类并把它放在新的文件夹结构下即可。在某个路径下只获取文件夹列表或者文件列表非常容易，因为我们用dirs和files分别维护了两个Dict。
    def __init__(self):
        self.root = Dir()

    # 这个指令中，我们初始化一个文件夹指针dir_struct，它指向根文件夹。我们将输入文件夹路径以'/'为依据划分开并得到每一层文件夹名字的数组path_list。然后我们遍历文件树并更新dir_struct指针让其指向当前的文件所代表的树节点。最后，我们要么停在最后一个文件夹里，或者停在一个文件上。如果输入最后一级是文件名字，我们只需要返回这个文件名。如果最后一级是文件夹，我们从它的dirs哈希表中得到所有子文件夹的名字。类似的，我们可以从最后一个文件夹的files哈希表中得到所有文件的列表。我们将两个列表接起来，排序后返回。时间 O(m+n+klog(k)), m表示输入字符串的长度, n 是 ls 输入参数的最深文件夹的深度, k是输入文件夹路径最后一层文件夹的条目数目（包括文件和子文件夹）. 空间为O(k)。
    def ls(self, path: str) -> List[str]:
        dir_struct = self.root
        files = []
        if path != "/":
            path_list = path.split("/")
            # loop deeper until the last dir
            for i in range(1, len(path_list) - 1):
                dir_struct = dir_struct.dirs[path_list[i]]
            # 我们要么停在最后一个文件夹里，或者停在一个文件上。
            if path_list[-1] in dir_struct.files:
                # 最后一级是文件名字，我们只需要返回这个文件名。
                files.append(path_list[-1])
                return files
            else:
                # 最后一级是文件夹
                dir_struct = dir_struct.dirs[path_list[-1]]
        # 我们从最后一级得到所有子文件夹的名字，和所有文件的列表
        files = list(dir_struct.dirs.keys()) + list(dir_struct.files.keys())
        files.sort()
        return files


    # 如ls, 我们从根文件夹开始逐层遍历文件树。一旦我们在mkdir中遇到一个不存在的文件夹,我们在最后一个文件夹的dirs结构中创建一个新的条目并把它的子文件夹列表初始化为空列表。我们持续这一过程直到我们到达最后一级目录。时间 O(m+n),n是最后一层文件夹的深度。空间为O(1)
    def mkdir(self, path: str) -> None:
        dir_struct = self.root
        path_list = path.split("/")
        for i in range(1, len(path_list)):
            # 一旦我们在mkdir中遇到一个不存在的文件夹,我们在最后一个文件夹的dirs结构中创建一个新的条目并把它的子文件夹列表初始化为空列表。
            if path_list[i] not in dir_struct.dirs:
                dir_struct.dirs[path_list[i]] = Dir()
            # 持续这一过程直到我们到达最后一级目录。
            dir_struct = dir_struct.dirs[path_list[i]]
   
    # 如ls, 我们逐层遍历文件路径。当我们到达文件所在文件夹时，我们检查files列表中是否已经存在file文件。如果文件已经存在，我们把当前内容添加到该文件的末尾，在对应文件的value部分添加。如果文件不存在，我们在当前文件夹的files列表中添加一个新的条目并将它的内容初始化为当前内容。时间 O(m+n),n是最后一层文件夹的深度。空间为O(1)
    def addContentToFile(self, filePath: str, content: str) -> None:
        dir_struct = self.root
        path_list = filePath.split("/")
        for i in range(1, len(path_list) - 1):
            # 逐层遍历文件路径
            dir_struct = dir_struct.dirs[path_list[i]]
        # 当我们到达文件所在文件夹时，我们检查files列表中是否已经存在file文件。
        if path_list[-1] in dir_struct.files:
            # 如果文件已经存在，我们把当前内容添加到该文件的末尾，在对应文件的value部分添加
            dir_struct.files[path_list[-1]] = dir_struct.files[path_list[-1]] + content
        else:
            # 如果文件不存在，我们在当前文件夹的files列表中添加一个新的条目并将它的内容初始化为当前内容
            dir_struct.files[path_list[-1]] = "" + content


    # 我们通过逐层遍历到达最后一层文件夹目录。在最后一层文件夹中，我们在dir_struct.files中找到文件名的条目，然后返回它对应的文件内容。时间 O(m+n),n是最后一层文件夹的深度。空间为O(1) 
    def readContentFromFile(self, filePath: str) -> str:
        dir_struct = self.root
        path_list = filePath.split("/")
        for i in range(1, len(path_list) - 1):
            # 逐层遍历文件路径
            dir_struct = dir_struct.dirs[path_list[i]]
        # 在最后一层文件夹中，找到文件名的条目, 返回它对应的文件内容
        file_content = dir_struct.files[path_list[-1]]
        return file_content

340. 至多包含 K 个不同字符的最长子串

给你一个字符串 s 和一个整数 k ，请你找出 至多 包含 k 个 不同 字符的最长子串，并返回该子串的长度。

 

示例 1：

输入：s = "eceba", k = 2
输出：3
解释：满足题目要求的子串是 "ece" ，长度为 3 。

示例 2：

输入：s = "aa", k = 1
输出：2
解释：满足题目要求的子串是 "aa" ，长度为 2 。

 

提示：

    1 <= s.length <= 5 * 104
    0 <= k <= 50


159. 至多包含两个不同字符的最长子串
给你一个字符串 s ，请你找出 至多 包含 两个不同字符 的最长子串，并返回该子串的长度。

 

示例 1：

输入：s = "eceba"
输出：3
解释：满足题目要求的子串是 "ece" ，长度为 3 。

示例 2：

输入：s = "ccaabbb"
输出：5
解释：满足题目要求的子串是 "aabbb" ，长度为 5 。

 

提示：

    1 <= s.length <= 105
    s 由英文字母组成



方法 1：滑动窗口 + 哈希表

想法

我们使用这个简单问题至多包含两个不同字符的最长子串的方法。

为了通过一次遍历解决这个问题，我们使用滑动窗口方法，使用两个指针 left 和 right 标记窗口的边界。

思路是将左右指针都设置为 0，然后向右移动 right 指针保证区间内含有不超过 k 个不同字符。当移动到含有 k + 1 个不同字符的时候，移动 left 指针直到区间内不含有超过 k + 1 个不同字符。

340-1.png

这个算法的基本思想是：在字符串上移动滑动窗口，保证窗口内有不超过 k 个不同字符，同时在每一步更新最大子串长度。

    只有一个问题需要解决：如何移动左指针保证子串中只包含 k 个不同字符？

我们可以使用一个哈希表，建立从字符到滑动窗口最右出现位置的映射，在任意时刻，哈希表不能包含 k+1 个元素。

340-2.png

例如，使用哈希表可以知道字符 O 在 "LOVELEE" 窗口最右出现位置为 1，所以只需要将 left 指针移动到位置 1 + 1 = 2 保证将字符 O 排除在滑动窗口外。

算法

算法实现如下：

    如果字符串为空或者 k 是零的话返回 0。
    设置指针为字符串开头 left = 0 和 right = 0，初始化最大子串长度 max_len = 1。
    当 right 指针小于 N 时：
        将当前字符 s[right] 加入哈希表并且向右移动 right 指针。
        如果哈希表包含 k + 1 个不同字符，在哈希表中移去最左出现的字符，移动 left 指针使得滑动窗口只包含 k 个不同字符。
        更新 max_len。

from collections import defaultdict
class Solution:
    def lengthOfLongestSubstringKDistinct(self, s: 'str', k: 'int') -> 'int':
        n = len(s) 
        if k == 0 or n == 0:
            return 0
        
        # sliding window left and right pointers
        left, right = 0, 0
        # hashmap character -> its rightmost position 
        # in the sliding window
        hashmap = defaultdict()

        max_len = 1
        
        while right < n:
            # add new character and move right pointer
            hashmap[s[right]] = right
            right += 1

            # slidewindow contains 3 characters
            if len(hashmap) == k + 1:
                # delete the leftmost character
                del_idx = min(hashmap.values())
                del hashmap[s[del_idx]]
                # move left pointer of the slidewindow
                left = del_idx + 1

            max_len = max(max_len, right - left)

        return max_len

复杂度分析

    使用了至多包含两个不同字符的最长子串问题的解法，我们可能达到最好的时间复杂度 O(N)O(N)O(N) 嘛？

对于最好情况，如果字符串不超过 k 个不同字符，答案是肯定的。因为只需要一次遍历就可以得到结果，时间复杂度是 O(N)O(N)O(N)。

对于最坏情况，当输入字符串包含 n 个不同字符，答案是不能。因为每一步都需要花费 O(k)O(k)O(k) 时间找到哈希表中的最小值，所以总的复杂度是 O(Nk)O(Nk)O(Nk)。

    时间复杂度：最好情况下是 O(N)O(N)O(N)，最坏情况下是 O(Nk)O(Nk)O(Nk)。
    空间复杂度：哈希表的空间开销：O(k)O(k)O(k)。

方法 2：滑动窗口 + 有序字典

如何达到 O(N)O(N)O(N) 时间复杂度

方法 1 使用了标准的哈希表，不能够保证 O(N)O(N)O(N) 的复杂度。

为了达到 O(N)O(N)O(N) 的效率，我们可以使用一种数据结构，保证以下四种操作都可以在 O(1)O(1)O(1) 时间完成：

    插入键
    获取键 / 检查键是否存在
    删除键
    返回最先 / 最后插入的键值对

前三个操作通过标准的哈希表就可以实现，第四个操作使用链表可以实现。

    使用有序字典结构，可以同时支持哈希表和链表操作，在 Python 中这个结构叫做 OrderedDict， Java 中为 LinkedHashMap。

有序字典在面试中很常见，相关例题可以查看LRU缓存机制问题。

算法

使用有序字典代替标准哈希表解决方法 1：

    如果字符串为空或者 k 是零的话返回 0。
    设置指针为字符串开头 left = 0 和 right = 0，初始化最大子串长度 max_len = 1。
    当 right 指针小于 N 时：
        如果当前字符 s[right] 已经在有序字典中 hashmap 中，删除它，以保证 hashmap 的第一个元素是滑动窗口的最左侧元素。
        将 s[right] 加入有序字典，并右移 right 指针。
        如果有序字典 hashmap 包含 k + 1 个不同字符，在哈希表中移去最左出现的字符，移动 left 指针使得滑动窗口只包含 k 个不同字符。
        更新 max_len。

from collections import OrderedDict
class Solution:
    def lengthOfLongestSubstringKDistinct(self, s: 'str', k: 'int') -> 'int':
        n = len(s) 
        if k == 0 or n == 0:
            return 0
        
        # sliding window left and right pointers
        left, right = 0, 0
        # hashmap character -> its rightmost position 
        # in the sliding window
        hashmap = OrderedDict()

        max_len = 1
        
        while right < n:
            character = s[right]
            # if character is already in the hashmap -
            # delete it, so that after insert it becomes
            # the rightmost element in the hashmap
            if character in hashmap:
                del hashmap[character]
            hashmap[character] = right
            right += 1

            # slidewindow contains k + 1 characters
            if len(hashmap) == k + 1:
                # delete the leftmost character
                _, del_idx = hashmap.popitem(last = False)
                # move left pointer of the slidewindow
                left = del_idx + 1

            max_len = max(max_len, right - left)

        return max_len

复杂度分析

    时间复杂度：O(N)O(N)O(N) 因为有序字典的所有操作 insert/get/delete/popitem
    (put/containsKey/remove) 都在常数时间内完成。
    空间复杂度：O(k)O(k)O(k)，有序字典的空间开销，最多保存 k + 1 个元素。

作者：LeetCode
链接：https://leetcode.cn/problems/longest-substring-with-at-most-k-distinct-characters/solution/zhi-shao-bao-han-k-ge-bu-tong-zi-fu-de-zui-chang-z/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

Python两种方法配详细注解。

# 暴力+剪枝 O(n^2)时间，5%速度

    def lengthOfLongestSubstringKDistinct(self, s: str, k: int) -> int:
        n = len(s)
        if n <= k: return n
        if k == 0: return 0
        max_len = k
        for i in range(n):
            if n - i + 1 <= max_len: break # 剪枝
            for j in range(i + max_len, n): # 至少max_len长度
                unique_c = len(set(s[i:j + 1]))
                if unique_c <= k: # 如果满足条件
                    max_len = max(max_len, j + 1 - i)
                else: 
                    break # 如果不满足，及时break

        return max_len

# 滑动窗口, 97% 时间：最坏O(n * k)。乘以k是遍历字典时的复杂度。 空间：O(k)。字典的额外空间。

    def lengthOfLongestSubstringKDistinct(self, s: str, k: int) -> int:
        n = len(s)
        if n <= k: return n
        if k == 0: return 0
        max_len = k
        
        mp = {} # 键对值存放每个字母最后一次出现时的index
        l = 0 # 窗口的左界
        for r in range(n): # 遍历窗口的右界
            if s[r] not in mp and len(mp) == k: # 如果mp里没有当前元素，且mp长度已满
                # max_len = max(max_len, r - l) # 更新已有window的宽度。注意不是r + 1 - l，因为此时r在当前window的后一位
                min_val = sys.maxsize
                for key, val in mp.items(): # 遍历一次字典（长度为k），找到最靠前的index（最小的val）
                    if val < min_val:
                        min_key, min_val = key, val
                del mp[min_key] # 删除这个最小index对应的key
                l = min_val + 1 # 让l指向mp里最小index对应的后一位（相当于window左界往前移动）             
            
            mp[s[r]] = r # 刷新当前元素最后一次出现时的index
            max_len = max(max_len, r + 1 - l) # 更新max_len

        return max_len



1246. 删除回文子数组

给你一个整数数组 arr，每一次操作你都可以选择并删除它的一个 回文 子数组 arr[i], arr[i+1], ..., arr[j]（ i <= j）。

注意，每当你删除掉一个子数组，右侧元素都会自行向前移动填补空位。

请你计算并返回从数组中删除所有数字所需的最少操作次数。

 

示例 1：

输入：arr = [1,2]
输出：2

示例 2：

输入：arr = [1,3,4,1,5]
输出：3
解释：先删除 [4]，然后删除 [1,3,1]，最后再删除 [5]。

 

提示：

    1 <= arr.length <= 100
    1 <= arr[i] <= 20



动态规划 dp[i][j]代表arr[i:j+1]的最短次数，

class Solution:
    def minimumMoves(self, arr: List[int]) -> int:
        n=len(arr)
        dp=[[1 for i in range(n)] for j in range(n)]#dp[k][k]=1
        val_idx={}
        for gap in range(1,n):
            for i in range(n):
                if i+gap>=n:
                    break
                j=i+gap
                dp[i][j]=float('inf') if arr[i]!=arr[j] else dp[i+1][j-1]
                for k in range(i,j):
                    if arr[i]==arr[k]:#k可以解释为分割位置
                        dp[i][j]=min(dp[i][j],dp[i][k]+dp[k+1][j])
        return dp[0][n-1]


解题思路

作者：megurine
链接：https://leetcode.cn/problems/palindrome-removal/solution/biao-zhun-qu-jian-dpyu-chu-li-shi-shi-ji-9ey0/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。
class Solution:
    def minimumMoves(self, arr: List[int]) -> int:
        n = len(arr)
        f = [[n] * n for i in range(n)]
        g = [None] * n 
        for i, x in enumerate(arr):
            f[i][i] = 1
            # 预处理找到 [i, n) 范围内所有与 a[i] 相等的值的下标，减少无效遍历
            g[i] = [j for j in range(i, n) if x == arr[j]]

        for i in range(n - 2, -1, -1):
            for j in range(i + 1, n):
                if i == j - 1:
                    f[i][j] = int(arr[i] != arr[j]) + 1
                else:
                    if arr[i] == arr[j]:
                        f[i][j] = f[i + 1][j - 1]
                    for k in g[i]:
                        if k >= j:
                            break
                        f[i][j] = min(f[i][j], f[i][k] + f[k + 1][j])

        return f[0][n - 1]


解题思路

滑动窗口，窗口大小逐渐增大。
dp[i][j]表示以下标i为起始点，以下标j为终点（包含）的数组的最小删除次数
当两端i,j相同时，需要删去的值更新为dp[i+1][j-1]。
但是两端相同时不一定分割的数量最少，如1,2,3,2,1,1,2,1
两端虽然相同，但是删去两端后，剩下2，3，2，1，1，2，还需要删3次。
所以还要遍历内部所有点，找到合适的分割点
代码

class Solution:
    def minimumMoves(self, arr: List[int]) -> int:
        n=len(arr)
        dp=[[2**32 for i in range(n)] for j in range(n)]
        for i in range(n):
            dp[i][i] = 1
        for gap in range(1,n):
            for i in range(n):
                if i+gap>=n:
                    break
                j=i+gap
                if gap == 1:
                    if arr[i] == arr[j]:
                        dp[i][j]= 1
                    else:
                        dp[i][j] = 2
                else:
                    if arr[i] == arr[j]:
                        dp[i][j] = dp[i+1][j-1]
                    for k in range(i,j):
                        dp[i][j]=min(dp[i][j],dp[i][k]+dp[k+1][j])
        return dp[0][n-1]


作者：handsomechen-c
链接：https://leetcode.cn/problems/palindrome-removal/solution/dong-tai-gui-hua-by-handsomechen-c-wx2r/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

252. 会议室

给定一个会议时间安排的数组 intervals ，每个会议时间都会包括开始和结束的时间 intervals[i] = [starti, endi] ，请你判断一个人是否能够参加这里面的全部会议。

 

示例 1：

输入：intervals = [[0,30],[5,10],[15,20]]
输出：false

示例 2：

输入：intervals = [[7,10],[2,4]]
输出：true

 

提示：

    0 <= intervals.length <= 104
    intervals[i].length == 2
    0 <= starti < endi <= 106



方法一：暴力法

最简单的方法是将数组中的会议全部两两比较，判断它们是否有冲突（即它们的时间是否有交叠）。若一个会议开始时另一个会议依然没有结束，则它们存在交叠。

class Solution {

  public boolean canAttendMeetings(int[][] intervals) {
    for (int i = 0; i < intervals.length; i++) {
      for (int j = i + 1; j < intervals.length; j++) {
        if (overlap(intervals[i], intervals[j]))
          return false;
      }
    }
    return true;
  }

  public static boolean overlap(int[] i1, int[] i2) {
    return ((i1[0] >= i2[0] && i1[0] < i2[1]) || (i2[0] >= i1[0] && i2[0] < i1[1]));
  }
}

交叠的情况

可以更简洁地编写上述代码中的交叠条件。考虑两个不交叠的会议。前面的会议在后面的会议开始之前结束。因此，两次会议的 最小 结束时间（即前面会议的结束时间）小于或等于两次会议的 最大 开始时间（即后面会议的开始时间)。

Two non-overlapping intervals

图 1. 两个不交叠的会议

Two overlapping intervals

图 2. 两个交叠的会议

因此可以这样重写：

public static boolean overlap(int[] i1, int[] i2) {
    return (Math.min(i1[1], i2[1]) >
            Math.max(i1[0], i2[0]));
}

复杂度分析

    时间复杂度 : O(n2)O(n^2)O(n2) 。我们对每两个会议都进行了比较。
    空间复杂度 : O(1)O(1)O(1) 。没有使用额外空间。

方法二：排序

思路是按照开始时间对会议进行排序。接着依次遍历会议，检查它是否在下个会议开始前结束。

class Solution {

  public boolean canAttendMeetings(int[][] intervals) {
    Arrays.sort(intervals, new Comparator<int[]>() {
      public int compare(int[] i1, int[] i2) {
        return i1[0] - i2[0];
      }
    });

    for (int i = 0; i < intervals.length - 1; i++) {
      if (intervals[i][1] > intervals[i + 1][0])
        return false;
    }
    return true;
  }
}

复杂度分析

    时间复杂度 : O(nlog⁡n)O(n \log n)O(nlogn) 。时间复杂度由排序决定。一旦排序完成，只需要 O(n)O(n)O(n) 的时间来判断交叠。
    空间复杂度 : O(1)O(1)O(1)。没有使用额外空间。

作者：LeetCode
链接：https://leetcode.cn/problems/meeting-rooms/solution/hui-yi-shi-by-leetcode/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

求解思路

    按照会议起始时间排序
    判断上一个会议的截止时间与下一个会议的起始时间是否交叉， intervals[i-1][1] <= intervals[i][0]

代码

class Solution:
    def canAttendMeetings(self, intervals: List[List[int]]) -> bool:
        intervals.sort()
        return all(intervals[i-1][1] <= intervals[i][0]  for i in range(1, len(intervals)))

复杂度分析

作者：100000000000001
链接：https://leetcode.cn/problems/meeting-rooms/solution/by-100000000000001-cyqk/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

解题思路

解法1：暴力排序，然后判断相邻的区间是否有交集，如果有说明不可以完成所有的会议，若都没有，说明可以。O(NlgN)时间。

解法2：不排序，按照区间处理，每当进入一个区间[a, b]，就把区间[a, b)内的conut累计1。最后扫描看看有没有时间点会议数目超过2，若有说明不能完成所有会议。否则可以。O(Nlg(Span)), N是数组intervals的长度，Span是所有区间内跨度最大的区间的跨度。

解法3 差分法： 在2的基础上的优化。不需要按顺序依次累计区间[a,b]内所有点，因为会议最多的时候一定能够通过检查起点获得。因此我们可以在进入一个区间的时候对区间start++，出来的时候对区间end--。最后再扫描一遍数组，看有没有时间点会议数目超过2，若有说明不能完成所有会议。否则可以

解法4：差分法2： 也是差分，就是相比于解法3用的数组表示，改用map表示，这样对于离散化（discreted）和稀疏数组（sparse）更高效。注意必须用ordered map （即C++中的map）而不能用unordered_map，因为最后的遍历需要按区间起点从小到大遍历。而且注意是end--，而不是 (end-1)-- 因为该题目允许刚参加完一个会议，立即参加下一个会议，是合法的。比如 [0, 5], [5, 10]是合法的。

作者：jyj407
链接：https://leetcode.cn/problems/meeting-rooms/solution/zhong-gui-zhong-ju-si-chong-bu-tong-de-j-ayuh/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。


class Solution:
    def canAttendMeetings(self, intervals: List[List[int]]) -> bool:
        mp = {}
        maxVal = 0
        mp = defaultdict(lambda:0, mp) 
        for interval in intervals:
            mp[interval[0]] += 1
            mp[interval[1]] -= 1
            maxVal = max(maxVal, interval[1])
        
      
        maxOverlap = 0
        for i in range(0, maxVal):
            maxOverlap += mp[i]

            if (maxOverlap > 1):
                return False
        
        return True


426. 将二叉搜索树转化为排序的双向链表

将一个 二叉搜索树 就地转化为一个 已排序的双向循环链表 。

对于双向循环列表，你可以将左右孩子指针作为双向循环链表的前驱和后继指针，第一个节点的前驱是最后一个节点，最后一个节点的后继是第一个节点。

特别地，我们希望可以 就地 完成转换操作。当转化完成以后，树中节点的左指针需要指向前驱，树中节点的右指针需要指向后继。还需要返回链表中最小元素的指针。

 

示例 1：

输入：root = [4,2,5,1,3] 


输出：[1,2,3,4,5]

解释：下图显示了转化后的二叉搜索树，实线表示后继关系，虚线表示前驱关系。

示例 2：

输入：root = [2,1,3]
输出：[1,2,3]

示例 3：

输入：root = []
输出：[]
解释：输入是空树，所以输出也是空链表。

示例 4：

输入：root = [1]
输出：[1]

 

提示：

    -1000 <= Node.val <= 1000
    Node.left.val < Node.val < Node.right.val
    Node.val 的所有值都是独一无二的
    0 <= Number of Nodes <= 2000



## 思路:

其实就是左右节点指向改变，右节点指向下一个比它大的数，左节点指向比它小的数。又是二叉搜索树，自然想到中序遍历。

思路一：中序遍历（非递归和递归）

写法一：用栈(非递归)

"""
# Definition for a Node.
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
"""
class Solution:
    def treeToDoublyList(self, root: 'Node') -> 'Node':
        if not root:return 
        # 当一个中间节点
        head = Node(-1, None, None)
        # 记录为先前节点,找到下一个节点才能串起来
        prev = head
        # 中序遍历的非递归
        stack = []
        p = root
        while p or stack:
            while p:
                stack.append(p)
                p = p.left
            p = stack.pop()
            # 改变左右方向
            prev.right = p
            p.left = prev
            # 改变先前节点
            prev = p
            p = p.right
        # 将head 删掉   
        head.right.left = prev
        prev.right = head.right
        return head.right

写法二：递归

"""
# Definition for a Node.
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
"""
class Solution:
    def treeToDoublyList(self, root: 'Node') -> 'Node':
        if not root:return 
        # 当一个中间节点
        head = Node(-1, None, None)
        # 记录为先前节点,找到下一个节点才能串起来
        prev = head

        # 中序遍历的递归
        def inorder(root):
            nonlocal prev
            if not root:
                return 
            inorder(root.left)
            prev.right = root
            root.left = prev
            prev = prev.right
            inorder(root.right)
        
        inorder(root)
        # 将head 删掉   
        head.right.left = prev
        prev.right = head.right
        return head.right

思路二：分治

"""
# Definition for a Node.
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
"""
class Solution:
    def treeToDoublyList(self, root: 'Node') -> 'Node':
        if not root: return
        left = self.treeToDoublyList(root.left)
        right = self.treeToDoublyList(root.right)
        root.left = root
        root.right = root
        return self.connect(self.connect(left, root), right)

    def connect(self, node1, node2):
        if not (node1 and node2):
            return node1 or node2
        tail1, tail2 = node1.left, node2.left
        tail1.right = node2
        node2.left = tail1
        tail2.right = node1
        node1.left = tail2
        return node1



346. 数据流中的移动平均值

给定一个整数数据流和一个窗口大小，根据该滑动窗口的大小，计算其所有整数的移动平均值。

实现 MovingAverage 类：

    MovingAverage(int size) 用窗口大小 size 初始化对象。
    double next(int val) 计算并返回数据流中最后 size 个值的移动平均值。

 

示例：

输入：
["MovingAverage", "next", "next", "next", "next"]
[[3], [1], [10], [3], [5]]
输出：
[null, 1.0, 5.5, 4.66667, 6.0]

解释：
MovingAverage movingAverage = new MovingAverage(3);
movingAverage.next(1); // 返回 1.0 = 1 / 1
movingAverage.next(10); // 返回 5.5 = (1 + 10) / 2
movingAverage.next(3); // 返回 4.66667 = (1 + 10 + 3) / 3
movingAverage.next(5); // 返回 6.0 = (10 + 3 + 5) / 3

 

提示：

    1 <= size <= 1000
    -105 <= val <= 105
    最多调用 next 方法 104 次



class MovingAverage:

    def __init__(self, size: int):
        self.size = size
        self.data = []

    def next(self, val: int) -> float:
        if len(self.data) < self.size:
            self.data.append(val)
            return sum(self.data) / len(self.data)
        else:
            self.data.pop(0)
            self.data.append(val)
            return sum(self.data) / len(self.data)

方法一：数组或列表

我们可以用数组或列表来记录所有传入的值。然后从中取出对应的元素来计算平均值。
在这里插入图片描述
算法：

    我们初始化 queue 来存储数据流的数据和移动窗口 n 的大小。
    每次调用 next(val)，首先将 val 添加到 queue 中，然后我们从 queue 取最后 n 个元素计算平均值。

class MovingAverage:
    def __init__(self, size: int):
        self.size = size
        self.queue = []
        
    def next(self, val: int) -> float:
        size, queue = self.size, self.queue
        queue.append(val)
        # calculate the sum of the moving window
        window_sum = sum(queue[-size:])

        return window_sum / min(len(queue), size)

复杂度分析

    时间复杂度：O(N)\mathcal{O}(N)O(N)。其中 NNN 是移动窗口的大小，每次调用 next(val)，我们需要从 queue 中检索 NNN 个元素。
    空间复杂度：O(M)\mathcal{O}(M)O(M)，是 queue 的大小。

方法二：双端队列

我们可以比方法一使用更优的时间复杂度和空间复杂度。

我们会注意到并不需要存储数据流中的所有值，只需要数据流中的最后 n 个值。

根据移动窗口的定义，在每个步骤中，我们向窗口添加一个新元素，同时从窗口中删除第一个元素。这里，我们可以应用一种称为双端队列的数据结构（deque）来实现移动窗口，它在两端删除或添加元素将具有常数的时间复杂度（O(1)\mathcal{O}(1)O(1)）；使用双端队列，我们可以将空间复杂度降低到 O(N)\mathcal{O}(N)O(N)，其中 NNN 是移动窗口的大小。
在这里插入图片描述

其次，为了计算移动窗口元素的总和 sum，我们不需要遍历窗口的全部元素。

我们可以保留前一个移动窗口的总和 sum，然后为了得到新的移动窗口的总和，我们只需要 sum+=new_val,sum-=first_val，这样就可以得到新的总和，其中 new_val 为添加的值，first_val 为原移动窗口中的第一个值，这样可以将时间复杂度降低到常数。

算法：
下面是 Python 中 deque 的定义。我们在其他编程语言（如 Java）中也有类似的 deque 实现。

Deques 是堆栈和队列的泛化（名称读作 deck，是双端队列的缩写）。Deques 支持线程安全，可以在两端在时间复杂度为 O(1)O(1)O(1) 进行添加和删除元素。

根据前面说的，我们用 deque 替换队列，并添加一个新的变量 window_sum，在常数时间内计算移动窗口的和。

from collections import deque
class MovingAverage:
    def __init__(self, size: int):
        self.size = size
        self.queue = deque()
        # number of elements seen so far
        self.window_sum = 0
        self.count = 0
    
    def next(self, val: int) -> float:
        self.count += 1
        # calculate the new sum by shifting the window
        self.queue.append(val)
        tail = self.queue.popleft() if self.count > self.size else 0
        
        self.window_sum = self.window_sum - tail + val
        
        return self.window_sum / min(self.size, self.count)

复杂度分析

    时间复杂度：O(1)\mathcal{O}(1)O(1)。
    空间复杂度：O(N)\mathcal{O}(N)O(N)，移动窗口的大小。

方法三：基于数组的循环队列

除了 deque 之外，还可以应用另一种有趣的数据结构，称为循环队列 circular queue，它是一个环形的队列。
在这里插入图片描述

循环队列的主要优点是，通过向循环队列中添加新元素，它会自动丢弃最旧的元素。与 deque 不同，我们不需要显式地删除最旧的元素。

循环队列的另一个优点是，一个指针就足以跟踪队列的两端，不像 deque 那样，我们必须为每一端保留一个指针。

算法：

无需使用任何库，可以轻松实现具有固定大小数组的循环队列。关键是 head 和 tail 元素的关系，我们可以用以下公式：

tail=(head+1)mod  size\text{tail} = (\text{head} + 1) \mod \text{size} tail=(head+1)modsize

换句话说，tail 元素就在 head 元素的旁边。一旦我们向前移动 head，我们将覆盖前面的 tail 元素。
在这里插入图片描述

class MovingAverage:
    def __init__(self, size: int):
        self.size = size
        self.queue = [0] * self.size
        self.head = self.window_sum = 0
        # number of elements seen so far
        self.count = 0
    
    def next(self, val: int) -> float:
        self.count += 1
        # calculate the new sum by shifting the window
        tail = (self.head + 1) % self.size
        self.window_sum = self.window_sum - self.queue[tail] + val
        # move on to the next head
        self.head = (self.head + 1) % self.size
        self.queue[self.head] = val
        return self.window_sum / min(self.size, self.count)

复杂度分析

    时间复杂度：O(1)\mathcal{O}(1)O(1)。我们可以看到在 next(val) 函数中没有循环。
    空间复杂度：O(N)\mathcal{O}(N)O(N)，循环队列使用的大小。

作者：LeetCode
链接：https://leetcode.cn/problems/moving-average-from-data-stream/solution/shu-ju-liu-zhong-de-yi-dong-ping-jun-zhi-by-leetco/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

510. 二叉搜索树中的中序后继 II

给定一棵二叉搜索树和其中的一个节点 node ，找到该节点在树中的中序后继。如果节点没有中序后继，请返回 null 。

一个节点 node 的中序后继是键值比 node.val 大所有的节点中键值最小的那个。

你可以直接访问结点，但无法直接访问树。每个节点都会有其父节点的引用。节点 Node 定义如下：

class Node {
    public int val;
    public Node left;
    public Node right;
    public Node parent;
}

 

示例 1：

输入：tree = [2,1,3], node = 1
输出：2
解析：1 的中序后继结点是 2 。注意节点和返回值都是 Node 类型的。

示例 2：

输入：tree = [5,3,6,2,4,null,null,1], node = 6
输出：null
解析：该结点没有中序后继，因此返回 null 。

示例 3：

输入：tree = [15,6,18,3,7,17,20,2,4,null,13,null,null,null,null,null,null,null,null,9], node = 15
输出：17

示例 4：

输入：tree = [15,6,18,3,7,17,20,2,4,null,13,null,null,null,null,null,null,null,null,9], node = 13
输出：15

示例 5：

输入：tree = [0], node = 0
输出：null

 

提示：

    树中节点的数目在范围 [1, 104] 内。
    -105 <= Node.val <= 105
    树中各结点的值均保证唯一。

 

进阶：你能否在不访问任何结点的值的情况下解决问题?


 前驱和后继

    前驱：指的是中序遍历的上一个结点，或者说是比当前结点小的最大结点。
    后继：指的是中序遍历的下一个结点，或者说是比当前结点大的最小结点。

在这里插入图片描述
方法：迭代

这里可能有两种情况：

    node 结点有右孩子，且它的后继在树中相对较低的位置，为了找到后继，我们应该向右走一次，再尽可能的向左走。

在这里插入图片描述

    node 结点没有右孩子，则它的后继在树中相对较高的位置，为了找到后继，我们向上走到直到结点 tmp 的左孩子是 node 的父节点时，则 node 的后继为 tmp。若没有找到符合条件的结点说明该结点没有后继。

在这里插入图片描述

在这里插入图片描述

算法：

    若 node 结点有右孩子，则它的后继在树中相对较低的位置。我们向右走一次，再尽可能的向左走，返回最后所在的结点。
    若 node 结点没有右孩子，则它的后继在树中相对较高的位置。我们向上走到直到结点 tmp 的左孩子是 node 的父节点时，则 node 的后继为 tmp。

class Solution:
    def inorderSuccessor(self, node: 'Node') -> 'Node':
        # the successor is somewhere lower in the right subtree
        if node.right:
            node = node.right
            while node.left:
                node = node.left
            return node
        
        # the successor is somewhere upper in the tree
        while node.parent and node == node.parent.right:
            node = node.parent
        return node.parent

复杂度分析

    时间复杂度：O(H)\mathcal{O}(H)O(H)。其中 HHH 为数的高度。平均时间复杂度为 O(log⁡N)\mathcal{O}(\log N)O(logN)，最坏的事件复杂度为 O(N)\mathcal{O}(N)O(N)，其中 NNN 为树的结点数。
    空间复杂度：O(1)\mathcal{O}(1)O(1)，在计算的过程中没有使用额外空间。

作者：LeetCode
链接：https://leetcode.cn/problems/inorder-successor-in-bst-ii/solution/er-cha-sou-suo-shu-zhong-de-zhong-xu-hou-ji-ii-by-/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

708. 循环有序列表的插入

给定循环单调非递减列表中的一个点，写一个函数向这个列表中插入一个新元素 insertVal ，使这个列表仍然是循环非降序的。

给定的可以是这个列表中任意一个顶点的指针，并不一定是这个列表中最小元素的指针。

如果有多个满足条件的插入位置，你可以选择任意一个位置插入新的值，插入后整个列表仍然保持有序。

如果列表为空（给定的节点是 null），你需要创建一个循环有序列表并返回这个节点。否则。请返回原先给定的节点。

 

示例 1：

 

输入：head = [3,4,1], insertVal = 2
输出：[3,4,1,2]
解释：在上图中，有一个包含三个元素的循环有序列表，你获得值为 3 的节点的指针，我们需要向表中插入元素 2 。新插入的节点应该在 1 和 3 之间，插入之后，整个列表如上图所示，最后返回节点 3 。


示例 2：

输入：head = [], insertVal = 1
输出：[1]
解释：列表为空（给定的节点是 null），创建一个循环有序列表并返回这个节点。

示例 3：

输入：head = [1], insertVal = 0
输出：[1,0]

 

提示：

    0 <= Number of Nodes <= 5 * 104
    -106 <= Node.val, insertVal <= 106



 方法一：一次遍历

如果循环链表为空，则插入一个新节点并将新节点的 next\textit{next}next 指针指向自身，插入新节点之后得到只有一个节点的循环链表，该循环链表一定是有序的，将插入的新节点作为新的头节点返回。

如果循环链表的头节点的 next\textit{next}next 指针指向自身，则循环链表中只有一个节点，在头节点之后插入新节点，将头节点的 next\textit{next}next 指针指向新节点，将新节点的 next\textit{next}next 指针指向头节点，此时循环链表中有两个节点且一定是有序的，返回头节点。

如果循环链表中的节点数大于 111，则需要从头节点开始遍历循环链表，寻找插入新节点的位置，使得插入新节点之后的循环链表仍然保持有序。

用 curr\textit{curr}curr 和 next\textit{next}next 分别表示当前节点和下一个节点，初始时 curr\textit{curr}curr 位于 head\textit{head}head，next\textit{next}next 位于 head\textit{head}head 的下一个节点，由于链表中的节点数大于 111，因此 curr≠next\textit{curr} \ne \textit{next}curr​=next。遍历过程中，判断值为 insertVal\textit{insertVal}insertVal 的新节点是否可以在 curr\textit{curr}curr 和 next\textit{next}next 之间插入，如果符合插入要求则在 curr\textit{curr}curr 和 next\textit{next}next 之间插入新节点，否则将 curr\textit{curr}curr 和 next\textit{next}next 同时向后移动，直到找到插入新节点的位置或者遍历完循环链表中的所有节点。

遍历过程中，如果找到插入新节点的位置，则有以下三种情况：

    curr.val≤insertVal≤next.val\textit{curr}.\textit{val} \le \textit{insertVal} \le \textit{next}.\textit{val}curr.val≤insertVal≤next.val，此时新节点的值介于循环链表中的两个节点值之间，在 curr\textit{curr}curr 和 next\textit{next}next 之间插入新节点；

    curr.val>next.val\textit{curr}.\textit{val} > \textit{next}.\textit{val}curr.val>next.val 且 insertVal>curr.val\textit{insertVal} > \textit{curr}.\textit{val}insertVal>curr.val，此时 curr\textit{curr}curr 和 next\textit{next}next 分别是循环链表中的值最大的节点和值最小的节点，insertVal\textit{insertVal}insertVal 大于 curr\textit{curr}curr 的节点值，因此新节点应该在 curr\textit{curr}curr 的后面插入，即在 curr\textit{curr}curr 和 next\textit{next}next 之间插入新节点；

    curr.val>next.val\textit{curr}.\textit{val} > \textit{next}.\textit{val}curr.val>next.val 且 insertVal<next.val\textit{insertVal} < \textit{next}.\textit{val}insertVal<next.val，此时 curr\textit{curr}curr 和 next\textit{next}next 分别是循环链表中的值最大的节点和值最小的节点，insertVal\textit{insertVal}insertVal 小于 next\textit{next}next 的节点值，因此新节点应该在 next\textit{next}next 的前面插入，即在 curr\textit{curr}curr 和 next\textit{next}next 之间插入新节点。

如果遍历完循环链表中的所有节点之后仍然没有遇到上述三种情况，则循环链表中的所有节点值都相同，因此新节点插入循环链表中的任何位置仍可以使循环链表保持有序，此时仍可在 curr\textit{curr}curr 和 next\textit{next}next 之间插入新节点。

在 curr\textit{curr}curr 和 next\textit{next}next 之间插入新节点的方法是：用 node\textit{node}node 表示值为 insertVal\textit{insertVal}insertVal 的新节点，令 curr.next\textit{curr}.\textit{next}curr.next 指向 node\textit{node}node，令 node.next\textit{node}.\textit{next}node.next 指向 next\textit{next}next，即完成插入新节点的操作。

插入新节点之后，返回循环链表的头节点。

class Solution:
    def insert(self, head: 'Node', insertVal: int) -> 'Node':
        node = Node(insertVal)
        if head is None:
            node.next = node
            return node
        if head.next == head:
            head.next = node
            node.next = head
            return head
        curr = head
        next = head.next
        while next != head:
            if curr.val <= insertVal <= next.val:
                break
            if curr.val > next.val:
                if insertVal > curr.val or insertVal < next.val:
                    break
            curr = curr.next
            next = next.next
        curr.next = node
        node.next = next
        return head

复杂度分析

    时间复杂度：O(n)O(n)O(n)，其中 nnn 是链表的节点数。需要遍历链表一次寻找插入节点的位置，插入节点的时间是 O(1)O(1)O(1)。

    空间复杂度：O(1)O(1)O(1)。

作者：LeetCode-Solution
链接：https://leetcode.cn/problems/insert-into-a-sorted-circular-linked-list/solution/xun-huan-you-xu-lie-biao-de-cha-ru-by-le-tiow/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

 方法一：双指针迭代

尽管问题看起来很简单，但是要编写一个涵盖所有情况的解决方案其实不简单。

通常对于链表的问题，可以采用双指针迭代的方法，用两个指针遍历链表。

用两个指针遍历的原因之一是在单链表中的没有对前继节点的引用，因此我们用一个指针指向当前节点的前继节点。

我们使用 prev 和 curr 两个指针遍历循环链表，当我们找到合适的插入位置时，我们将其插入到 prev 和 curr 之间。

在这里插入图片描述

算法：

首先，我们定义双指针迭代的算法框架如下：

    我们用 prev 和 curr 两个指针循环遍历链表。循环的终止条件是两个指针返回到起点，即 prev==head。
    在每一次迭代过程中，我们都会检查由两个指针限定的位置是否是新值插入的正确位置。
    如果不是正确的位置，则两个指针都向前移动一步。现在困难的地方是如何设计出一个简洁的逻辑来处理解决不同的情况。我们分为三个不同的情况。

情况 1： 新节点的值位于当前链表的最小值和最大值之间。因此，应该将其插入到链表中间。

在这里插入图片描述

从上面的例子我们可以看到，新值（6）位于链表的最大值和最小值之间（即 1 和 9）之间。无论从哪个节点开始（在本例子中从节点 {3} 开始），新节点最终会被插入到节点 {5} 和 {7} 之间。

我们要找到满足 {prev.val <= insertVal <= curr.val} 条件的位置。

情况 2： 新值超出了链表中的最小值和最大值，即小于最小值或大于最大值。在任一情况下，新值都应插入在尾节点（即链表最大值的节点）之后。

在这里插入图片描述

在这里插入图片描述

首先我们要在链表中找到相邻节点值下降的位置，即 {prev.val>curr.val} 来找到尾节点。因为节点值是升序排序的，所以尾节点在链表中具有最大的值。

此外，检查新值是大于尾节点的值还是小于头节点的值，分别由 prev 和 curr 指向。

图片 case 2.1 对应于新值大于或等于尾节点值的条件，即 {insertVal>=prev.val}。

图片 case 2.2 对应于新值小于或等于头节点值的条件，即 {insertVal<=curr.val}。

一旦找到头尾节点，就可以通过在头尾节点之间插入新值来扩展链表，即在 prev 和 curr 之间插入新值。

情况 3： 链表的元素的值相同。
尽管在问题描述中没有说明，但是链表可能出现所有节点的值均相同。

在这里插入图片描述

在这种情况下，我们最终会遍历链表回到起点。然后我们可以在任一节点后插入新值，那我们不如在入口节点之后插入新值。

注意，我们不能够跳过迭代，因为必须遍历链表来确定链表是否只存在一个唯一的值。

还有一种情况，是当链表为空时，我们可以在进入循环之前解决它。

class Solution:
    def insert(self, head: 'Node', insertVal: int) -> 'Node':

        if head == None:
            newNode = Node(insertVal, None)
            newNode.next = newNode
            return newNode
 
        prev, curr = head, head.next
        toInsert = False

        while True:
            
            if prev.val <= insertVal <= curr.val:
                # Case #1.
                toInsert = True
            elif prev.val > curr.val:
                # Case #2. where we locate the tail element
                # 'prev' points to the tail, i.e. the largest element!
                if insertVal >= prev.val or insertVal <= curr.val:
                    toInsert = True

            if toInsert:
                prev.next = Node(insertVal, curr)
                # mission accomplished
                return head

            prev, curr = curr, curr.next
            # loop condition
            if prev == head:
                break
        # Case #3.
        # did not insert the node in the loop
        prev.next = Node(insertVal, curr)
        return head

复杂度分析

    时间复杂度：O(N)\mathcal{O}(N)O(N)。其中 NNN 指的时链表的元素个数。最坏的情况下我们会遍历整个链表。
    空间复杂度：O(1)\mathcal{O}(1)O(1)，仅仅使用了两个指针。

作者：LeetCode
链接：https://leetcode.cn/problems/insert-into-a-sorted-circular-linked-list/solution/xun-huan-you-xu-lie-biao-de-cha-ru-by-leetcode/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

Python3+链表
代码

"""
# Definition for a Node.
class Node:
    def __init__(self, val=None, next=None):
        self.val = val
        self.next = next
"""

class Solution:
    def insert(self, head: 'Optional[Node]', insertVal: int) -> 'Node':
        if not head:
            ans = Node(insertVal)
            ans.next = ans
            return ans

        cur = head
        lower = upper = head
        while cur:
            if cur.val < lower.val:
                lower = cur
            if cur.val > upper.val:
                upper = cur
            cur = cur.next
            if cur == head:
                break

        if insertVal < lower.val or insertVal > upper.val:
            cur = head
            while cur:
                if cur.val == upper.val and cur.next.val == lower.val:
                    node = Node(insertVal)
                    tmp = cur.next
                    cur.next = node
                    cur.next.next = tmp
                    return head
                cur = cur.next

        cur = head
        while cur:
            if cur.val <= insertVal <= cur.next.val:
                node = Node(insertVal)
                tmp = cur.next
                cur.next = node
                cur.next.next = tmp
                return head
            cur = cur.next

作者：liupengsay
链接：https://leetcode.cn/problems/insert-into-a-sorted-circular-linked-list/solution/er-xu-cheng-ming-jiu-xu-zui-by-liupengsa-8lhi/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

759. 员工空闲时间

给定员工的 schedule 列表，表示每个员工的工作时间。

每个员工都有一个非重叠的时间段  Intervals 列表，这些时间段已经排好序。

返回表示 所有 员工的 共同，正数长度的空闲时间 的有限时间段的列表，同样需要排好序。

示例 1：

输入：schedule = [[[1,2],[5,6]],[[1,3]],[[4,10]]]
输出：[[3,4]]
解释：
共有 3 个员工，并且所有共同的
空间时间段是 [-inf, 1], [3, 4], [10, inf]。
我们去除所有包含 inf 的时间段，因为它们不是有限的时间段。

 

示例 2：

输入：schedule = [[[1,3],[6,7]],[[2,4]],[[2,5],[9,12]]]
输出：[[5,6],[7,9]]

 

（尽管我们用 [x, y] 的形式表示 Intervals ，内部的对象是 Intervals 而不是列表或数组。例如，schedule[0][0].start = 1, schedule[0][0].end = 2，并且 schedule[0][0][0] 是未定义的）

而且，答案中不包含 [5, 5] ，因为长度为 0。

 

注：

    schedule 和 schedule[i] 为长度范围在 [1, 50]的列表。
    0 <= schedule[i].start < schedule[i].end <= 10^8。

注：输入类型于 2019 年 4 月 15 日 改变。请重置为默认代码的定义以获取新方法。



class Solution:
    def employeeFreeTime(self, schedule: '[[Interval]]') -> '[Interval]':
        from sortedcontainers import SortedList
        sl = SortedList()
        
        for item in schedule:
            for ele in item:
                sl.add((ele.start, ele.end))

        ans = []
        curMax = sl[0][1]
        for idx in range(1, len(sl)):
            
            if sl[idx][0] > curMax:
                tmp = Interval(curMax, sl[idx][0])
                ans.append(tmp)
            
            curMax = max(curMax, sl[idx][1])
        
        return ans
            

方法一：事件（扫描线）[通过]

如果某个区间与任一区间重叠，则该区间不会出现在答案中。所以我们可以将问题转换为：给定一组区间，找出所有员工都不包含的区间。

我们可以使用区间问题中的 “事件” 方法。对于每个区间 [s, e]，我们可以看作有两个事件：当 time = s 时，balance++；当 time = e 时，balance--。我们只关心 balance == 0 的区间。

算法：

对于每个区间，创建如上所述的两个事件，并对事件进行排序。在事件 t 发生的每个事件，如果 balance == 0，则说明 [prev，t] 是所有员工都不包含的区间，其中 prev 是 t 的前一个值。

class Solution(object):
    def employeeFreeTime(self, avails):
        OPEN, CLOSE = 0, 1

        events = []
        for emp in avails:
            for iv in emp:
                events.append((iv.start, OPEN))
                events.append((iv.end, CLOSE))

        events.sort()
        ans = []
        prev = None
        bal = 0
        for t, cmd in events:
            if bal == 0 and prev is not None:
                ans.append(Interval(prev, t))

            bal += 1 if cmd is OPEN else -1
            prev = t

        return ans

复杂度分析

    时间复杂度：O(Clog⁡C)O(C\log C)O(ClogC)，其中 CCC 是所有员工的区间数。
    空间复杂度：O(C)O(C)O(C)。

方法二：优先队列 [通过]

假设在某个时间段没有员工工作，这个时间段将持续到某个员工要工作时为止。

我们维护员工下一次要工作的数据。当处理完当前工作时，我们为该员工添加下一次的工作。

算法：

我们跟踪最新的时间 anchor。当我们处理尚未处理的工作时，时间为 t，员工 e_id，是该员工的第 e_jx 个工作。如果 anchor < t，则存在一个空闲区间 Interval(anchor, t)。

class Solution(object):
    def employeeFreeTime(self, avails):
        ans = []
        pq = [(emp[0].start, ei, 0) for ei, emp in enumerate(avails)]
        heapq.heapify(pq)
        anchor = min(iv.start for emp in avails for iv in emp)
        while pq:
            t, e_id, e_jx = heapq.heappop(pq)
            if anchor < t:
                ans.append(Interval(anchor, t))
            anchor = max(anchor, avails[e_id][e_jx].end)
            if e_jx + 1 < len(avails[e_id]):
                heapq.heappush(pq, (avails[e_id][e_jx+1].start, e_id, e_jx+1))

        return ans

复杂度分析

    时间复杂度：O(Clog⁡N)O(C\log N)O(ClogN)，其中 NNN 是员工的数量，CCC 是所有员工的区间数，数据堆的最大大小为 NNN，每个 pop 和 push 操作需要 O(log⁡N)O(\log N)O(logN)，有 O(C)O(C)O(C) 个这样的操作。
    空间复杂度：O(N)O(N)O(N)，使用了一个一维数组 dp。

作者：LeetCode
链接：https://leetcode.cn/problems/employee-free-time/solution/yuan-gong-kong-xian-shi-jian-by-leetcode/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

class Solution(object):
    def employeeFreeTime(self, avails):
        ans = []
        pq = [(emp[0].start, ei, 0) for ei, emp in enumerate(avails)]
        heapq.heapify(pq)
        anchor = min(s for s,_,v in pq)
        while pq:
            t, e_id, next = heapq.heappop(pq)
            if anchor < t:
                ans.append(Interval(anchor, t))
            if avails[e_id][next].end>anchor:
                anchor =avails[e_id][next].end
            w=next+1
            if w< len(avails[e_id]):
                heapq.heappush(pq, (avails[e_id][w].start, e_id, w))

        return ans
#作者：LeetCode


359. 日志速率限制器

请你设计一个日志系统，可以流式接收消息以及它的时间戳。每条 不重复 的消息最多只能每 10 秒打印一次。也就是说，如果在时间戳 t 打印某条消息，那么相同内容的消息直到时间戳变为 t + 10 之前都不会被打印。

所有消息都按时间顺序发送。多条消息可能到达同一时间戳。

实现 Logger 类：

    Logger() 初始化 logger 对象
    bool shouldPrintMessage(int timestamp, string message) 如果这条消息 message 在给定的时间戳 timestamp 应该被打印出来，则返回 true ，否则请返回 false 。

 

示例：

输入：
["Logger", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage"]
[[], [1, "foo"], [2, "bar"], [3, "foo"], [8, "bar"], [10, "foo"], [11, "foo"]]
输出：
[null, true, true, false, false, false, true]

解释：
Logger logger = new Logger();
logger.shouldPrintMessage(1, "foo");  // 返回 true ，下一次 "foo" 可以打印的时间戳是 1 + 10 = 11
logger.shouldPrintMessage(2, "bar");  // 返回 true ，下一次 "bar" 可以打印的时间戳是 2 + 10 = 12
logger.shouldPrintMessage(3, "foo");  // 3 < 11 ，返回 false
logger.shouldPrintMessage(8, "bar");  // 8 < 12 ，返回 false
logger.shouldPrintMessage(10, "foo"); // 10 < 11 ，返回 false
logger.shouldPrintMessage(11, "foo"); // 11 >= 11 ，返回 true ，下一次 "foo" 可以打印的时间戳是 11 + 10 = 21

 

提示：

    0 <= timestamp <= 109
    每个 timestamp 都将按非递减顺序（时间顺序）传递
    1 <= message.length <= 30
    最多调用 104 次 shouldPrintMessage 方法



方法一：队列 + 集合

在我们解决问题之前，必须明确问题的条件。这里有一个重要的注意事项：可能会有几条日志同时到达。

我们知道日志的顺序是按时间顺序排列的，即日志的时间戳是单调递增的，尽管不是严格地递增。这个条件非常关键，因为它可以简化任务，如下面的解决方案所示。

第一个解决方案，我们按照问题中描述的任务直观地实现一个解决方案。

我们将接收的日志存储在队列中。为了加速重复项的检查，我们使用集合来检索日志。

在这里插入图片描述

从上面的示例可以看出，其中数字表示每个日志的时间戳，时间戳为 18 的日志到达后将使时间戳为 5 和 7 的日志失效，这两个日志超出了 10 秒的时间窗口。

算法：

    首先，我们使用队列作为滑动窗口，将存储所有时间范围内（10 秒）的日志。
    在接收每个日志后，它都有一个时间戮，这意味着滑动窗口需要变化，所以应该将队列中超出时间范围的日志失效。
    由于队列和集合应该同步，我们还应该在集合中删除失效的日志。
    更新队列和集合后，我们只需检查新接收的日志是否有任何重复项。如果没有，我们将日志添加到队列和集合中。

from collections import deque

class Logger(object):

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self._msg_set = set()
        self._msg_queue = deque()
    
    def shouldPrintMessage(self, timestamp, message):
        """
        Returns true if the message should be printed in the given timestamp, otherwise returns false.
        """
        while self._msg_queue:
            msg, ts = self._msg_queue[0]
            if timestamp - ts >= 10:
                self._msg_queue.popleft()
                self._msg_set.remove(msg)
            else:
                break
        
        if message not in self._msg_set:
            self._msg_set.add(message)
            self._msg_queue.append((message, timestamp))
            return True
        else:
            return False

可以发现使用集合并不是绝对必要的。也可以简单地迭代队列来检查是否有任何重复的日志。

另一个重要的注意事项是如果消息不是按时间顺序排列的，那么我们必须遍历整个队列以删除过期的日志。可以使用一些排序队列（如优先级队列）来保存日志。

复杂度分析

    时间复杂度：O(N)\mathcal{O}(N)O(N)。NNN 是队列的大小，在最坏的情况下，队列中的日志全部过期了，则需要情况队列。
    空间复杂度：O(N)\mathcal{O}(N)O(N)，NNN 是队列的大小，我们在队列和集合中存储了日志的信息，所需空间的上限是 2N2N2N，当没有重复的日志时。

方法二：哈希表 / 字典

我们可以将队列和集合数据结构组合成一个哈希表或字典，这样我们就能够保留队列中所有唯一的日志，也能够快速判断日志的重复性。

做法是用一个哈希表 / 字典，日志作为键，时间戳作为值。哈希表保留所有唯一的日志以及日志的最新时间戳。

在这里插入图片描述
从上面的例子可以看到，哈希表中有一条记录，日志 m2 和时间戳 2。然后出现另一条日志 m2，时间戳为 15。由于日志是在 13 秒之前打印的（即超出时间范围），因此它可以再次打印消息。然后，日志 m2 的时间戳将更新为 15。

算法：

    我们初始化一个哈希表 / 字典来记录日志到时间戳的映射。
    新日志到达时，条件如下时可以打印：
        我们以前从未见过这个日志。
        我们以前见过这条日志但是是在 10 多秒之前。
    在上述两种情况下，我们将更新哈希表中对应的日志的时间戮。

class Logger(object):

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self._msg_dict = {}
    
    def shouldPrintMessage(self, timestamp, message):
        """
        Returns true if the message should be printed in the given timestamp, otherwise returns false.
        """
        if message not in self._msg_dict:
            # case 1). add the message to print
            self._msg_dict[message] = timestamp
            return True

        if timestamp - self._msg_dict[message] >= 10:
            # case 2). update the timestamp of the message
            self._msg_dict[message] = timestamp
            return True
        else:
            return False

注：为了清楚起见，我们把这两个情况分别处理，也可以把这两个情况结合起来，得到一个更简洁的解决方案。

这种使用哈希表的方法与使用队列的方法的主要区别在于，在队列中我们要进行清理，即在每次调用函数时，我们首先删除那些过期的日志。

在哈希表中即使日志过期，我们也会保留所有日志。这种特性可能会成为问题，因为随着时间的推移，内存的使用将继续增长。有时，更需要使用前面方法的垃圾回收机制。

复杂度分析

    时间复杂度：O(1)O(1)O(1)。哈希表的查找和更新是常数的时间复杂度。
    空间复杂度：O(M)O(M)O(M)，其中 MMM 是日志的数量。随着时间的推移，哈希表记录了每个出现过的唯一日志。

作者：LeetCode
链接：https://leetcode.cn/problems/logger-rate-limiter/solution/ri-zhi-su-lu-xian-zhi-qi-by-leetcode/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

哈希表,不多bb.
代码

class Logger:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.val = dict()


    def shouldPrintMessage(self, timestamp: int, message: str) -> bool:
        """
        Returns true if the message should be printed in the given timestamp, otherwise returns false.
        If this method returns false, the message will not be printed.
        The timestamp is in seconds granularity.
        """
        if not message in self.val: 
            self.val[message] = timestamp + 10
            return True
        else:
            if timestamp < self.val[message]: return False
            else:
                self.val[message] = timestamp + 10 # 记得更新哈希表
                return True

作者：yun6688
链接：https://leetcode.cn/problems/logger-rate-limiter/solution/pythonjie-fa-ri-zhi-su-lu-xian-zhi-qi-by-4fi9/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

158. 用 Read4 读取 N 个字符 II

给你一个文件 file ，并且该文件只能通过给定的 read4 方法来读取，请实现一个方法使其能够使 read 读取 n 个字符。注意：你的 read 方法可能会被调用多次。

read4 的定义：

read4 API 从文件中读取 4 个连续的字符，然后将这些字符写入缓冲区数组 buf4 。

返回值是读取的实际字符数。

请注意，read4() 有其自己的文件指针，类似于 C 中的 FILE * fp 。

    参数类型: char[] buf4
    返回类型: int

注意: buf4[] 是目标缓存区不是源缓存区，read4 的返回结果将会复制到 buf4[] 当中。

下列是一些使用 read4 的例子：

File file("abcde"); // 文件名为 "abcde"， 初始文件指针 (fp) 指向 'a' 
char[] buf4 = new char[4]; // 创建一个缓存区使其能容纳足够的字符
read4(buf4); // read4 返回 4。现在 buf4 = "abcd"，fp 指向 'e'
read4(buf4); // read4 返回 1。现在 buf4 = "e"，fp 指向文件末尾
read4(buf4); // read4 返回 0。现在 buf4 = ""，fp 指向文件末尾

 

read 方法：

通过使用 read4 方法，实现 read 方法。该方法可以从文件中读取 n 个字符并将其存储到缓存数组 buf 中。您 不能 直接操作 file 。

返回值为实际读取的字符。

read 的定义：

    参数类型:  char[] buf, int n
    返回类型:  int

注意: buf[] 是目标缓存区不是源缓存区，你需要将结果写入 buf[] 中。

注意：

    你 不能 直接操作该文件，文件只能通过 read4 获取而 不能 通过 read。
    read  函数可以被调用 多次。
    请记得 重置 在 Solution 中声明的类变量（静态变量），因为类变量会 在多个测试用例中保持不变，影响判题准确。请 查阅 这里。
    你可以假定目标缓存数组 buf 保证有足够的空间存下 n 个字符。 
    保证在一个给定测试用例中，read 函数使用的是同一个 buf。

 

示例 1：

输入： file = "abc"， queries = [1,2,1]
输出：[1,2,0]
解释：测试用例表示以下场景:
File file("abc");
Solution sol;
sol.read (buf, 1); // 调用 read 方法后，buf 应该包含 “a”。我们从文件中总共读取了 1 个字符，所以返回 1。
sol.read (buf, 2); // 现在 buf 应该包含 "bc"。我们从文件中总共读取了 2 个字符，所以返回 2。
sol.read (buf, 1); // 我们已经到达文件的末尾，不能读取更多的字符。所以返回 0。
假设已经分配了 buf ，并保证有足够的空间存储文件中的所有字符。

示例 2：

输入：file = "abc"， queries = [4,1]
输出：[3,0]
解释：测试用例表示以下场景:
File file("abc");
Solution sol;
sol.read (buf, 4); // 调用 read 方法后，buf 应该包含 “abc”。我们从文件中总共读取了 3 个字符，所以返回 3。
sol.read (buf, 1); // 我们已经到达文件的末尾，不能读取更多的字符。所以返回 0。

 

提示：

    1 <= file.length <= 500
    file 由英语字母和数字组成
    1 <= queries.length <= 10
    1 <= queries[i] <= 500



Python 简单直观的解法】这道理我觉得难就难在读题，我第一次做这类题硬是花了半小时读题才弄明白它要我做什么。注意几个点：

    题意总结起来一句话就是要你用read4去实现读取任意长度的字符，就是这样，那个buf只是储存单次read读出来的字符，并不是累计所有read读出来的字符！！ 我一开始以为buf是存累计read读取的所有字符的，题目没有说清楚。 举个例子，如果n=7，那就是用read4去读取7个字符, buf里面放的就是这7个字符。至于是哪7个字符，那就要看上一次的read读到哪个字符，从上一次的位置往下读7个字符
    测试案例中，read是可以多次调用的！！这就意味着需要记录上一次read读到了哪个位置，可以理解为有一个指针指向最后一个已读字符的位置。
    read4只能读4个字符，n可能大于4也可能小于4. 即使n=1，调用一次read4也会读取4个字符，即便我们只需要返回第一个字符，而下一次调用read4的时候就是从第5个字符开始读了。因此，我们需要额外的空间去把每一次read4读取的字符存起来。
    唯一能知道当前读到哪个位置的方法就是通过read4。

class Solution:
    def __init__(self):
        self.buf4 = ['']*4
        self.cache = [] #记录累计调用read4读取的字符中未通过read读取的字符，有点绕。比如read4累计调用了4次，共读取16个字符， 但是第一次read的n=3， 第二次read的n=10，累计read已读13个字符，此时cache还剩16-13=3个字符未读

    def read(self, buf: List[str], n: int) -> int:
        while n > len(self.cache):
            cnt = read4(self.buf4)
            if cnt > 0:
                self.cache.extend(self.buf4[:cnt]) #先把read4读取的字符放入缓存
            else:
                break
        toRead = len(self.cache[:n]) #需要读取的字符数量，如果此时self.cache的length < n, python会返回self.cache的length
        buf[:toRead] = self.cache[:toRead] #把这需要读取的字符写入buf
        self.cache = self.cache[toRead:] #修改cache，因为我们刚刚又从cache读取了toRead个字符，所以只需要保留剩下的字符
        return toRead


157. 只调用一次read

cursor记录当前buf更新的位置，也是read4一共读取的字符数，循环读取字符直接更新到buf里，一旦当前read4读到的字符数小于4（file读完了）或者n本来就小于等于4（不管怎样都只需要读一次read4）就停止循环，否则循环直到cursor大于等于n。

最终读取的字符数不是n就是cursor，取其中小的那一个。严谨一点的话，最后buf里不应该有多余的字符。

class Solution:
    def read(self, buf, n):
        cur = 0
        while cur < n:
            res = [''] * 4
            num = read4(res)
            buf[cur: cur + num] = res
            cur += num
            if num < 4 or n <= 4:
                break 
        return min(n, cur)

158. 多次调用read

多次调用read4，每次都会接着上一次的位置读取，所以需要一个类变量缓存每次多余的字符。

每次调用read时先检查上一次多余的字符够不够，够的话直接更新buf和缓存并返回n。cursor表示缓存里字符总数。循环向缓存读字符，停止条件和之前一样，最后更新buf和缓存，返回cursor和n中小的那一个。

class Solution:
    def __init__(self):
        self.buf = []

    def read(self, buf, n):
        buf_len = len(self.buf)

        if len(self.buf) >= n:
            buf[:n] = self.buf[:n]
            self.buf = self.buf[n:]
            return n
        
        cur = buf_len
        while cur < n:
            cache = [''] * 4
            num = read4(cache)
            self.buf += cache[:num]
            cur += num
            if num < 4 or n <= 4:
                break
        
        res = min(cur, n)
        buf[:res] = self.buf[:res]
        self.buf = self.buf[res:]
        return res

作者：imkk
链接：https://leetcode.cn/problems/read-n-characters-given-read4-ii-call-multiple-times/solution/157-158-read4du-qu-nzi-fu-by-imkk/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

694. 不同岛屿的数量

给定一个非空 01 二维数组表示的网格，一个岛屿由四连通（上、下、左、右四个方向）的 1 组成，你可以认为网格的四周被海水包围。

请你计算这个网格中共有多少个形状不同的岛屿。两个岛屿被认为是相同的，当且仅当一个岛屿可以通过平移变换（不可以旋转、翻转）和另一个岛屿重合。

 

示例 1：

输入: grid = [[1,1,0,0,0],[1,1,0,0,0],[0,0,0,1,1],[0,0,0,1,1]]
输出：1

示例 2：

输入: grid = [[1,1,0,1,1],[1,0,0,0,0],[0,0,0,0,1],[1,1,0,1,1]]
输出: 3

 

提示：

    m == grid.length
    n == grid[i].length
    1 <= m, n <= 50
    grid[i][j] 仅包含 0 或 1


official page has no solution.LOL
https://leetcode.cn/problems/number-of-distinct-islands/solution/bu-tong-dao-yu-de-shu-liang-by-leetcode/


python代码如下：

class Solution:
    def numDistinctIslands(self, grid: List[List[int]]) -> int:
        shapes = set()
        m, n = len(grid), len(grid[0])
        for i in range(m):
            for j in range(n):
                if grid[i][j] == 1:
                    shape = []
                    self.dfs(grid, m, n, i, j, 0, 0, shape)
                    shapes.add(''.join(shape))
        return len(shapes)
    
    def dfs(self, grid, m, n, i, j, sumX, sumY, shape):
        shape.append(str(sumX) + str(sumY))
        grid[i][j] = 0
        dirs = [[-1, 0], [0, -1], [1, 0], [0, 1]]
        for r, c in dirs:
            x = i + r
            y = j + c
            sumX += r
            sumY += c
            if x >= 0 and x < m and y >= 0 and y < n and grid[x][y] == 1:
                self.dfs(grid, m, n, x, y, sumX, sumY, shape)
        return



class Solution:
    def numDistinctIslands(self, grid: List[List[int]]) -> int:
        m,n = len(grid),len(grid[0])
        vis = [[0] * n for _ in range(m)]
        l = [[0,1],[0,-1],[1,0],[-1,0]]
        a = []
        def dfs(i,j):
            vis[i][j] = 1
            a.append([i,j])
            for dx,dy in l:
                x,y = i+dx,j+dy
                if 0<=x<m and 0<=y<n and vis[x][y]==0 and grid[x][y]==1:
                    dfs(x,y)
            return 
        def draw(a):
            a.sort()
            t1 = a[0][0]
            a.sort(key = lambda v:v[1])
            t2 = a[0][1]
            s = []
            for i,j in a:
                s.append((i-t1)*n+j-t2)
            return tuple(s)
        ss = set()
        for i in range(m):
            for j in range(n):
                if vis[i][j]==0 and grid[i][j]==1:
                    dfs(i,j)
                    
                    ss.add(draw(a))                
                    a = []
        print(ss)
        return len(ss)

772. 基本计算器 III

实现一个基本的计算器来计算简单的表达式字符串。

表达式字符串只包含非负整数，算符 +、-、*、/ ，左括号 ( 和右括号 ) 。整数除法需要 向下截断 。

你可以假定给定的表达式总是有效的。所有的中间结果的范围均满足 [-231, 231 - 1] 。

注意：你不能使用任何将字符串作为表达式求值的内置函数，比如 eval() 。

 

示例 1：

输入：s = "1+1"
输出：2

示例 2：

输入：s = "6-4/2"
输出：4

示例 3：

输入：s = "2*(5+5*2)/3+(6/2+8)"
输出：21

 

提示：

    1 <= s <= 104
    s 由整数、'+'、'-'、'*'、'/'、'(' 和 ')' 组成
    s 是一个 有效的 表达式



 解题思路

双栈法，stack_num记录数字，stack_opt记录操作符。首先来看一种最基本的计算操作：stack_num中pop出来两个数A,B, stack_opt中pop出来一个操作符opt, 计算结果为B opt A, 将结果存到stack_num中。

字符有以下几种情况：

    空格，直接下一个字符。
    数字，直接入stack_num栈。需要注意处理多位数。
    '(', 直接入stack_opt栈
    ')', 重复最基本的计算操作，直到stack_opt栈栈顶为')'。然后stack_opt栈pop出来')'。
    操作符，如果栈顶操作符的优先级大于等于当前操作符的优先级，则重复最基本的计算操作，直到stack_opt栈空或者其栈顶操作符的优先级小于当前操作符的优先级，再将当前操作符入stack_num栈。优先级定义priorty = {'(':0,')':0,'+':1,'-':1,'*':2,'/':2}。数字越大，优先级与越高。

在遍历完所有字符后，判断当前操作栈stack_opt是否为空,不为空，重复最基本的计算操作，知道操作栈为stack_opt为空。最后结果就是stack_num中剩余的唯一一个元素。
代码

class Solution:
    def calculate(self, s: str) -> int:
        stack_num = []
        stack_opt = []
        i = 0
        priorty = {'(':0,')':0,'+':1,'-':1,'*':2,'/':2}
        while i<len(s):
            if s[i]==' ':
                i += 1
                continue
            if '0'<=s[i]<='9':
                j = i
                while i+1<len(s) and '0'<=s[i+1]<='9':
                    i += 1
                num = int(s[j:i+1])
                stack_num.append(num)
            elif s[i]=='(':
                stack_opt.append(s[i])
            elif s[i]==')':
                while stack_opt[-1]!='(':
                    opt = stack_opt.pop()
                    A = stack_num.pop()
                    B = stack_num.pop()
                    res = self.calc(A,B,opt)
                    stack_num.append(res)
                stack_opt.pop()
            else:
                while stack_opt and priorty[stack_opt[-1]]>=priorty[s[i]]:
                    opt = stack_opt.pop()
                    A = stack_num.pop()
                    B = stack_num.pop()
                    res = self.calc(A,B,opt)
                    stack_num.append(res)
                stack_opt.append(s[i])
            i += 1

        while stack_opt:
            opt = stack_opt.pop()
            A = stack_num.pop()
            B = stack_num.pop()
            res = self.calc(A,B,opt)
            stack_num.append(res)
        return stack_num[-1]
    
    def calc(self,num1,num2,opt):
        if opt=='+':
            return int(num1)+int(num2)
        elif opt=='-':
            return int(num2)-int(num1)
        elif opt=='*':
            return int(num2)*int(num1)
        elif opt=='/':
            return int(num2)//int(num1)

作者：qianbaidu
链接：https://leetcode.cn/problems/basic-calculator-iii/solution/shuang-zhan-fa-tong-yi-jie-jue-ji-ben-ji-suan-qi-s/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

解题思路

详情参考：https://labuladong.github.io/algo/4/33/134/
需要注意的是在遇到 + - * / ) 以及len(s)==0 这六种情况下需要计算数值，并且赋值sgn为c
代码

class Solution():
    def calculate(self,s: str) -> int:
        def f(s):
            num,sgn,st=0,'+',deque()
            while len(s)>0:
                c = s.pop(0)
                if c.isdigit():
                    num = 10*num + int(c)
                if c=='(': num = f(s)
                if c in ['+','-','*','/',')'] or len(s)==0:
                    if sgn=='+': st.append(num)
                    elif sgn=='-': st.append(-num)
                    elif sgn=='*': st[-1] *= num
                    elif sgn=='/': st[-1] = int(st[-1]/num)
                    sgn,num = c,0
                if c==')': break
            return sum(st)
        return f(list(s))

作者：0djKioGGSD
链接：https://leetcode.cn/problems/basic-calculator-iii/solution/ke-neng-shi-zui-jian-dan-de-dai-ma-by-0d-4kh0/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

思路和心得：

1.和DSA书上的模拟过程一样

不过书上用了一个栈，c++必须两个栈，因为元素类型会卡

2.注意要收尾

class Solution:
    def calculate(self, s: str) -> int:
        #双栈是为了照顾c++  DSA书上的正统的解法
        #专门反馈了问题，“向下截断”====“向零取整”
        n = len(s)
        priority = {'(': 0, ')': 0, '+': 1, '-': 1, '*': 2, '/': 2}
        opt_stk = []
        num_stk = []
        i = 0
        while i < n:
            if s[i] == ' ':
                i += 1
                continue
            elif s[i].isdigit() != 0:   #把当前这个数字找完
                num = int(s[i])
                while i+1 < n and s[i+1].isdigit() != 0:
                    num = num * 10 + int(s[i+1])
                    i += 1
                num_stk.append(num)
            elif s[i] == '(':
                opt_stk.append(s[i])
            elif s[i] == ')':           #把当前这个括号里的东西算完
                while opt_stk and opt_stk[-1] != '(':
                    opt = opt_stk.pop(-1)
                    y = num_stk.pop(-1)
                    x = num_stk.pop(-1)
                    z = self.calc(x, y, opt)
                    num_stk.append(z)
                opt_stk.pop()
            else :                      #是运算符号 加减之前，先把乘除计算完
                while opt_stk and priority[opt_stk[-1]] >= priority[s[i]]:
                    opt = opt_stk.pop(-1)
                    y = num_stk.pop(-1)
                    x = num_stk.pop(-1)
                    z = self.calc(x, y, opt)
                    num_stk.append(z)
                opt_stk.append(s[i])
                
            i += 1

        while opt_stk:              #收尾
            opt = opt_stk.pop(-1)
            y = num_stk.pop(-1)
            x = num_stk.pop(-1)
            z = self.calc(x, y, opt)
            num_stk.append(z)
        
        return num_stk[-1]

    def calc (self, x: int, y: int, opt: chr) -> int:
        if opt == '+':
            return x + y
        elif opt == '-':
            return x - y
        elif opt == '*':
            return x * y
        else:
            if x / y < 0:
                return ceil(x/y)
            else:
                return floor(x/y)


作者：code_learner
链接：https://leetcode.cn/problems/basic-calculator-iii/solution/c-python3-dsashu-shang-de-jie-shi-fu-you-ss75/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。